In [ ]:
# @title Cell 1: Environment Setup and Data Loading
# Notebook: 03_Proposed_GWO_XGBoost.ipynb
# Purpose: Initialize environment, load fixed splits, configure checkpoint system

import warnings
warnings.filterwarnings('ignore')

import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path

# Machine Learning Libraries
import xgboost as xgb
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss
)
from sklearn.preprocessing import LabelEncoder

# Visualization
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend for server environment
import matplotlib.pyplot as plt
import seaborn as sns

# Set global configurations
np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

print("=" * 80)
print("PROPOSED METHOD: GWO-XGBOOST FOR ASD DETECTION")
print("Notebook 03: Grey Wolf Optimizer with XGBoost Wrapper")
print("=" * 80)
print()

# ============================================================================
# @ SECTION 1: Mount Google Drive and Initialize Paths
# ============================================================================

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

BASE_DIR = Path('/content/drive/MyDrive')
DATA_DIR = BASE_DIR / 'ASD_GWO_XGBoost_Project' / '01_Dataset' / 'splits' / 'no_ethnicity' / 'preprocessed'
RESULTS_DIR = BASE_DIR / '03_Results'
BASELINE_DIR = RESULTS_DIR / 'baseline_experiments'
PROPOSED_DIR = RESULTS_DIR / 'proposed_method'

# Create directory structure
CHECKPOINT_DIR = PROPOSED_DIR / 'checkpoints'
PARAM_TUNING_DIR = PROPOSED_DIR / 'parameter_tuning'
FINAL_EXP_DIR = PROPOSED_DIR / 'final_experiments'
MODELS_DIR = PROPOSED_DIR / 'models'
VIZ_DIR = PROPOSED_DIR / 'visualizations'

for directory in [CHECKPOINT_DIR, PARAM_TUNING_DIR, FINAL_EXP_DIR, MODELS_DIR, VIZ_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print("# Directory Initialization")
print(f"Base Directory: {BASE_DIR}")
print(f"Data Directory: {DATA_DIR}")
print(f"Results Directory: {PROPOSED_DIR}")
print(f"Checkpoint Directory: {CHECKPOINT_DIR}")
print()

# ============================================================================
# @ SECTION 2: Load Fixed Data Splits
# ============================================================================

print("# Loading Fixed Data Splits")
print("-" * 80)

# Load training set
train_df = pd.read_csv(DATA_DIR / 'train_set_preprocessed.csv')
X_train = train_df.drop('ASD_traits', axis=1)
y_train = train_df['ASD_traits']

# Load validation set
val_df = pd.read_csv(DATA_DIR / 'val_set_preprocessed.csv')
X_val = val_df.drop('ASD_traits', axis=1)
y_val = val_df['ASD_traits']

# Load test set
test_df = pd.read_csv(DATA_DIR / 'test_set_preprocessed.csv')
X_test = test_df.drop('ASD_traits', axis=1)
y_test = test_df['ASD_traits']

print(f"Training Set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Validation Set: {X_val.shape[0]} samples, {X_val.shape[1]} features")
print(f"Test Set: {X_test.shape[0]} samples, {X_test.shape[1]} features")
print()

# ============================================================================
# @ SECTION 3: Data Integrity Verification
# ============================================================================

print("# Data Integrity Verification")
print("-" * 80)

# Check for expected dimensions
assert X_train.shape[0] == 1270, f"Expected 1270 training samples, got {X_train.shape[0]}"
assert X_val.shape[0] == 318, f"Expected 318 validation samples, got {X_val.shape[0]}"
assert X_test.shape[0] == 397, f"Expected 397 test samples, got {X_test.shape[0]}"
assert X_train.shape[1] == 25, f"Expected 25 features, got {X_train.shape[1]}"

print("Split verification: PASSED")
print(f"  - Training: 1270 samples (64.0%)")
print(f"  - Validation: 318 samples (16.0%)")
print(f"  - Test: 397 samples (20.0%)")
print(f"  - Total: {X_train.shape[0] + X_val.shape[0] + X_test.shape[0]} samples")
print()

# Verify feature consistency
assert list(X_train.columns) == list(X_val.columns) == list(X_test.columns), \
    "Feature columns mismatch across splits"

# Check for prohibited columns
prohibited_cols = ['CASE_NO_PATIENT\'S', 'Ethnicity']
for col in prohibited_cols:
    assert col not in X_train.columns, f"Prohibited column found: {col}"

print("Feature verification: PASSED")
print(f"  - No prohibited columns (CASE_NO_PATIENT'S, Ethnicity)")
print(f"  - Consistent features across splits")
print()

# Target distribution
train_asd_pct = (y_train == 1).mean() * 100
val_asd_pct = (y_val == 1).mean() * 100
test_asd_pct = (y_test == 1).mean() * 100

print("Target Distribution:")
print(f"  - Training: {train_asd_pct:.1f}% ASD, {100-train_asd_pct:.1f}% Non-ASD")
print(f"  - Validation: {val_asd_pct:.1f}% ASD, {100-val_asd_pct:.1f}% Non-ASD")
print(f"  - Test: {test_asd_pct:.1f}% ASD, {100-test_asd_pct:.1f}% Non-ASD")
print()

# Feature list
feature_names = X_train.columns.tolist()
n_features = len(feature_names)

print(f"Feature List (n={n_features}):")
behavioral_features = [f for f in feature_names if f.startswith('A')]
clinical_features = [f for f in feature_names if f in ['CARS', 'AQ10', 'SRS']]
demographic_features = [f for f in feature_names if f not in behavioral_features + clinical_features]

print(f"  - Behavioral (n={len(behavioral_features)}): {', '.join(behavioral_features)}")
print(f"  - Clinical Scores (n={len(clinical_features)}): {', '.join(clinical_features)}")
print(f"  - Demographic (n={len(demographic_features)}): {', '.join(demographic_features)}")
print()

# ============================================================================
# @ SECTION 4: Checkpoint Manager Implementation
# ============================================================================

print("# Checkpoint Manager Configuration")
print("-" * 80)

class GWOCheckpointManager:
    """
    Hierarchical checkpoint manager for GWO optimization phases.

    Strategy:
    - Coarse Grid: Save after all 45 experiments complete
    - Final Runs: Save after EACH run (1-10) for maximum safety

    Attributes:
        phase (str): Optimization phase ('coarse' or 'final')
        checkpoint_dir (Path): Directory for checkpoint storage
        save_every (int): Save frequency (unused for current strategy)
    """

    def __init__(self, phase, checkpoint_dir, save_every=1):
        self.phase = phase
        self.checkpoint_dir = Path(checkpoint_dir)
        self.save_every = save_every
        self.checkpoint_file = self.checkpoint_dir / f'{phase}_checkpoint.json'

    def should_save(self, current_idx, total):
        """Determine if checkpoint should be saved at current index."""
        if self.phase == 'final':
            return True  # Save after every final run
        elif self.phase == 'coarse':
            return (current_idx + 1) == total  # Save only at completion
        return False

    def save(self, data):
        """Save checkpoint data to JSON file."""
        checkpoint_data = {
            'phase': self.phase,
            'timestamp': datetime.now().isoformat(),
            'data': data
        }

        with open(self.checkpoint_file, 'w') as f:
            json.dump(checkpoint_data, f, indent=2)

        print(f"Checkpoint saved: {self.checkpoint_file.name}")

    def load(self):
        """Load checkpoint data if exists."""
        if self.checkpoint_file.exists():
            with open(self.checkpoint_file, 'r') as f:
                checkpoint_data = json.load(f)

            print(f"Checkpoint loaded: {self.checkpoint_file.name}")
            print(f"  - Phase: {checkpoint_data['phase']}")
            print(f"  - Timestamp: {checkpoint_data['timestamp']}")

            return checkpoint_data['data']

        return None

    def cleanup(self):
        """Remove checkpoint file after successful completion."""
        if self.checkpoint_file.exists():
            self.checkpoint_file.unlink()
            print(f"Checkpoint cleaned: {self.checkpoint_file.name}")

# Initialize checkpoint managers
coarse_checkpoint_mgr = GWOCheckpointManager('coarse', CHECKPOINT_DIR)
final_checkpoint_mgr = GWOCheckpointManager('final', CHECKPOINT_DIR)

print("Checkpoint managers initialized:")
print(f"  - Coarse Grid: Save at completion (45/45 experiments)")
print(f"  - Final Runs: Save after each run (1-10)")
print()

# ============================================================================
# @ SECTION 5: Configuration and Metadata
# ============================================================================

print("# Experiment Configuration")
print("-" * 80)

# XGBoost configuration for fitness evaluation
XGBOOST_FITNESS_CONFIG = {
    'n_estimators': 150,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 1.0,
    'colsample_bytree': 1.0,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'eval_metric': 'auc',
    'random_state': 42,
    'objective': 'binary:logistic',
    'verbosity': 0
}

# XGBoost configuration for final model training
XGBOOST_FINAL_CONFIG = {
    'n_estimators': 500,
    'max_depth': 8,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'eval_metric': 'auc',
    'random_state': 42,
    'objective': 'binary:logistic',
    'verbosity': 0
}

# GWO configuration
GWO_CONFIG = {
    'min_features': 8,
    'max_features': 15,
    'random_state': 42
}

# Evaluation configuration
EVAL_CONFIG = {
    'primary_metric': 'roc_auc',
    'secondary_metrics': ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'log_loss'],
    'averaging': 'macro'
}

print("XGBoost Configuration (Fitness):")
print(f"  - n_estimators: {XGBOOST_FITNESS_CONFIG['n_estimators']}")
print(f"  - max_depth: {XGBOOST_FITNESS_CONFIG['max_depth']}")
print(f"  - learning_rate: {XGBOOST_FITNESS_CONFIG['learning_rate']}")
print(f"  - GPU: {XGBOOST_FITNESS_CONFIG['tree_method']}")
print()

print("XGBoost Configuration (Final):")
print(f"  - n_estimators: {XGBOOST_FINAL_CONFIG['n_estimators']}")
print(f"  - max_depth: {XGBOOST_FINAL_CONFIG['max_depth']}")
print(f"  - learning_rate: {XGBOOST_FINAL_CONFIG['learning_rate']}")
print()

print("GWO Configuration:")
print(f"  - Feature constraints: [{GWO_CONFIG['min_features']}, {GWO_CONFIG['max_features']}]")
print(f"  - Random state: {GWO_CONFIG['random_state']}")
print()

print("Evaluation Configuration:")
print(f"  - Primary metric: {EVAL_CONFIG['primary_metric']}")
print(f"  - Averaging strategy: {EVAL_CONFIG['averaging']}")
print()

# ============================================================================
# @ SECTION 6: GPU Verification
# ============================================================================

print("# GPU Configuration Verification")
print("-" * 80)

try:
    # Test GPU availability
    import subprocess
    gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'])
    gpu_info = gpu_info.decode('utf-8').strip()
    print(f"GPU Detected: {gpu_info}")

    # Test XGBoost GPU
    test_clf = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        n_estimators=10,
        random_state=42
    )
    test_clf.fit(X_train.iloc[:100], y_train.iloc[:100])
    print("XGBoost GPU test: PASSED")

except Exception as e:
    print(f"WARNING: GPU not available or not configured properly")
    print(f"Error: {str(e)}")
    print("Falling back to CPU configuration")

    XGBOOST_FITNESS_CONFIG['tree_method'] = 'hist'
    XGBOOST_FITNESS_CONFIG['predictor'] = 'cpu_predictor'
    XGBOOST_FINAL_CONFIG['tree_method'] = 'hist'
    XGBOOST_FINAL_CONFIG['predictor'] = 'cpu_predictor'

print()

# ============================================================================
# @ SECTION 7: Save Setup Information
# ============================================================================

print("# Saving Setup Information")
print("-" * 80)

setup_info = {
    'metadata': {
        'notebook': '03_Proposed_GWO_XGBoost.ipynb',
        'author': 'Chatelia Dyah Prameswari',
        'student_id': '1301223320',
        'institution': 'Universitas Telkom - Program Studi Sarjana Informatika',
        'project_title': 'Deteksi Autism Spectrum Disorder pada Anak Menggunakan Pemodelan XGBoost dan Grey Wolf Optimizer',
        'timestamp': datetime.now().isoformat()
    },
    'data_configuration': {
        'data_directory': str(DATA_DIR),
        'train_samples': int(X_train.shape[0]),
        'val_samples': int(X_val.shape[0]),
        'test_samples': int(X_test.shape[0]),
        'n_features': int(n_features),
        'feature_names': feature_names,
        'target_distribution': {
            'train_asd_pct': float(train_asd_pct),
            'val_asd_pct': float(val_asd_pct),
            'test_asd_pct': float(test_asd_pct)
        }
    },
    'model_configuration': {
        'xgboost_fitness': XGBOOST_FITNESS_CONFIG,
        'xgboost_final': XGBOOST_FINAL_CONFIG,
        'gwo': GWO_CONFIG,
        'evaluation': EVAL_CONFIG
    },
    'directory_structure': {
        'base_dir': str(BASE_DIR),
        'results_dir': str(PROPOSED_DIR),
        'checkpoint_dir': str(CHECKPOINT_DIR),
        'param_tuning_dir': str(PARAM_TUNING_DIR),
        'final_exp_dir': str(FINAL_EXP_DIR),
        'models_dir': str(MODELS_DIR),
        'viz_dir': str(VIZ_DIR)
    },
    'optimization_strategy': {
        'approach': 'Hybrid Validation (Coarse + Direct Final)',
        'skip_fine_grid': True,
        'rationale': 'Time efficiency for TA S1 scope - 56% time saved with marginal performance trade-off',
        'phases': [
            {'name': 'Coarse Grid', 'experiments': 45, 'estimated_hours': 3},
            {'name': 'Final Experiments', 'runs': 10, 'estimated_hours': 2},
            {'name': 'Analysis', 'estimated_hours': 1}
        ],
        'total_estimated_hours': 6
    }
}

setup_info_path = PROPOSED_DIR / 'setup_info.json'
with open(setup_info_path, 'w') as f:
    json.dump(setup_info, f, indent=2)

print(f"Setup information saved: {setup_info_path.name}")
print()

# ============================================================================
# @ SECTION 8: Summary and Next Steps
# ============================================================================

print("=" * 80)
print("CELL 1 EXECUTION SUMMARY")
print("=" * 80)
print()
print("Status: COMPLETED SUCCESSFULLY")
print()
print("Verification Results:")
print(f"  - Data splits loaded: 3/3 (train, val, test)")
print(f"  - Feature count verified: {n_features} features")
print(f"  - Target distribution: Balanced across splits")
print(f"  - GPU configuration: {'ENABLED' if 'gpu' in XGBOOST_FITNESS_CONFIG['tree_method'] else 'DISABLED'}")
print(f"  - Checkpoint system: Initialized")
print(f"  - Directory structure: Created")
print()
print("Ready for Cell 2: GWO Core Implementation")
print("=" * 80)

PROPOSED METHOD: GWO-XGBOOST FOR ASD DETECTION
Notebook 03: Grey Wolf Optimizer with XGBoost Wrapper

Mounted at /content/drive
# Directory Initialization
Base Directory: /content/drive/MyDrive
Data Directory: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/01_Dataset/splits/no_ethnicity/preprocessed
Results Directory: /content/drive/MyDrive/03_Results/proposed_method
Checkpoint Directory: /content/drive/MyDrive/03_Results/proposed_method/checkpoints

# Loading Fixed Data Splits
--------------------------------------------------------------------------------
Training Set: 1270 samples, 25 features
Validation Set: 318 samples, 25 features
Test Set: 397 samples, 25 features

# Data Integrity Verification
--------------------------------------------------------------------------------
Split verification: PASSED
  - Training: 1270 samples (64.0%)
  - Validation: 318 samples (16.0%)
  - Test: 397 samples (20.0%)
  - Total: 1985 samples

Feature verification: PASSED
  - No prohibited columns 

In [ ]:
# @title Cell 2: Grey Wolf Optimizer Core Implementation
# Notebook: 03_Proposed_GWO_XGBoost.ipynb
# Purpose: Implement pure GWO algorithm for binary feature selection
# Author: Chatelia Dyah Prameswari (1301223320)
# Institution: Universitas Telkom - Program Studi Sarjana Informatika

import numpy as np
import json
from pathlib import Path
from datetime import datetime
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("CELL 2: GREY WOLF OPTIMIZER CORE IMPLEMENTATION")
print("Implementation: Pure GWO with Random Constraint Enforcement")
print("=" * 80)
print()

# ============================================================================
# @ SECTION 1: Output Directory Configuration
# ============================================================================

print("# Output Directory Configuration")
print("-" * 80)

# Define output directory
OUTPUT_DIR = Path('/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_2_pureGWO')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Create subdirectories
CONFIG_DIR = OUTPUT_DIR / 'configurations'
TEST_DIR = OUTPUT_DIR / 'validation_tests'
MODEL_DIR = OUTPUT_DIR / 'gwo_models'

for directory in [CONFIG_DIR, TEST_DIR, MODEL_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"Output Directory: {OUTPUT_DIR}")
print(f"  - Configurations: {CONFIG_DIR}")
print(f"  - Validation Tests: {TEST_DIR}")
print(f"  - GWO Models: {MODEL_DIR}")
print()

# ============================================================================
# @ SECTION 2: Binary Conversion Utility
# ============================================================================

print("# Binary Conversion Implementation")
print("-" * 80)

def binary_conversion(continuous_position, threshold=0.5):
    """
    Convert continuous position to binary feature selection vector.

    Implementation: Simple threshold-based conversion
    Reference: Emary et al. (2016) - Binary Grey Wolf Optimization

    Args:
        continuous_position (np.ndarray): Continuous position vector [0,1]
        threshold (float): Conversion threshold (default: 0.5)

    Returns:
        np.ndarray: Binary vector (0 or 1)
    """
    binary_position = np.where(continuous_position >= threshold, 1, 0)
    return binary_position.astype(int)


def validate_threshold_effectiveness(position_history):
    """
    Post-optimization sanity check for threshold conversion quality.

    Metrics:
    1. Clarity Score: Mean distance from ambiguous boundary (0.5)
       - Excellent: >0.35 (decisive positions)
       - Good: 0.25-0.35
       - Acceptable: 0.15-0.25
       - Problematic: <0.15 (ambiguous)

    2. Oscillation Percentage: Feature flip rate in last 10 iterations
       - Excellent: <10%
       - Good: 10-20%
       - Acceptable: 20-30%
       - Problematic: >30%

    Args:
        position_history (list): List of position arrays per iteration

    Returns:
        dict: Validation metrics
    """
    if len(position_history) < 10:
        return {
            'clarity_score': None,
            'oscillation_pct': None,
            'status': 'insufficient_data'
        }

    # Metric 1: Position clarity (distance from 0.5)
    final_positions = position_history[-1]
    clarity_score = np.mean(np.abs(final_positions - 0.5))

    # Metric 2: Oscillation analysis (last 10 iterations)
    last_10_binary = [binary_conversion(pos) for pos in position_history[-10:]]
    flip_counts = np.zeros(len(final_positions))

    for i in range(len(last_10_binary) - 1):
        flips = (last_10_binary[i] != last_10_binary[i+1]).astype(int)
        flip_counts += flips

    oscillation_pct = (flip_counts > 0).sum() / len(final_positions)

    # Determine status
    if clarity_score > 0.35 and oscillation_pct < 0.10:
        status = 'excellent'
    elif clarity_score > 0.25 and oscillation_pct < 0.20:
        status = 'good'
    elif clarity_score > 0.15 and oscillation_pct < 0.30:
        status = 'acceptable'
    else:
        status = 'problematic'

    return {
        'clarity_score': float(clarity_score),
        'oscillation_pct': float(oscillation_pct),
        'status': status,
        'interpretation': {
            'clarity': 'excellent' if clarity_score > 0.35 else 'good' if clarity_score > 0.25 else 'acceptable' if clarity_score > 0.15 else 'problematic',
            'oscillation': 'excellent' if oscillation_pct < 0.10 else 'good' if oscillation_pct < 0.20 else 'acceptable' if oscillation_pct < 0.30 else 'problematic'
        }
    }

print("Binary conversion utilities implemented:")
print("  - binary_conversion(): Simple threshold (>=0.5)")
print("  - validate_threshold_effectiveness(): Post-optimization sanity check")
print()

# ============================================================================
# @ SECTION 3: Constraint Enforcement - Pure Random
# ============================================================================

print("# Constraint Enforcement Implementation")
print("-" * 80)

def enforce_constraints(binary_position, min_features, max_features, random_state=None):
    """
    Enforce feature selection constraints using random selection.

    Implementation: Pure GWO approach (unbiased random enforcement)

    Constraints:
    - Minimum features: 8
    - Maximum features: 15

    Strategy:
    - Too few (<8): Randomly activate features until minimum reached
    - Too many (>15): Randomly deactivate features until maximum reached
    - Within range [8,15]: No modification

    Args:
        binary_position (np.ndarray): Binary feature vector
        min_features (int): Minimum number of features (default: 8)
        max_features (int): Maximum number of features (default: 15)
        random_state (int): Random seed for reproducibility

    Returns:
        np.ndarray: Constraint-enforced binary vector
    """
    if random_state is not None:
        np.random.seed(random_state)

    binary_position = binary_position.copy()

    # CRITICAL FIX: Explicit type conversion to Python native int
    n_selected = int(np.sum(binary_position))

    # Case 1: Too few features
    if n_selected < min_features:
        zero_indices = np.where(binary_position == 0)[0]
        add_count = int(min_features - n_selected)  # Explicit int conversion

        if len(zero_indices) >= add_count:
            add_indices = np.random.choice(zero_indices, add_count, replace=False)
            binary_position[add_indices] = 1
        else:
            # Edge case: not enough features to add
            binary_position[zero_indices] = 1

    # Case 2: Too many features
    elif n_selected > max_features:
        one_indices = np.where(binary_position == 1)[0]
        remove_count = int(n_selected - max_features)  # Explicit int conversion

        if len(one_indices) >= remove_count:
            remove_indices = np.random.choice(one_indices, remove_count, replace=False)
            binary_position[remove_indices] = 0
        else:
            # Edge case: constraint cannot be satisfied (should not happen)
            # Keep as is and log warning
            pass

    # Case 3: Within range - no modification needed

    return binary_position

print("Constraint enforcement implemented:")
print("  - Strategy: Pure Random (unbiased GWO approach)")
print("  - Range: [8, 15] features")
print("  - Method: Random add/remove to satisfy constraints")
print("  - Bug Fix: Explicit type conversion for numpy scalars")
print()

# ============================================================================
# @ SECTION 4: Fitness Function
# ============================================================================

print("# Fitness Function Implementation")
print("-" * 80)

def fitness_function(feature_subset, X_train, y_train, X_val, y_val,
                    feature_names, xgb_config, verbose=False):
    """
    Evaluate feature subset quality using XGBoost validation AUC.

    Implementation: Wrapper method with XGBoost classifier
    Primary Metric: ROC-AUC on validation set

    Edge Cases Handled:
    1. Empty feature subset: Return 0.0 (worst fitness)
    2. XGBoost training failure: Return 0.5 (random guess baseline)
    3. AUC < 0.5: Clip to 0.5 (sanity check)

    Args:
        feature_subset (np.ndarray): Binary vector (25-bit)
        X_train (pd.DataFrame): Training features
        y_train (pd.Series): Training labels
        X_val (pd.DataFrame): Validation features
        y_val (pd.Series): Validation labels
        feature_names (list): List of feature names
        xgb_config (dict): XGBoost hyperparameters
        verbose (bool): Print debug information

    Returns:
        float: Validation AUC score [0.0, 1.0]
    """
    # Edge case 1: Empty feature subset
    selected_indices = np.where(feature_subset == 1)[0]

    if len(selected_indices) == 0:
        if verbose:
            print("  Warning: Empty feature subset, returning fitness=0.0")
        return 0.0

    # Select features
    selected_feature_names = [feature_names[i] for i in selected_indices]
    X_train_selected = X_train[selected_feature_names]
    X_val_selected = X_val[selected_feature_names]

    try:
        # Train XGBoost
        model = xgb.XGBClassifier(**xgb_config)
        model.fit(
            X_train_selected,
            y_train,
            eval_set=[(X_val_selected, y_val)],
            verbose=False
        )

        # Predict probabilities
        y_val_proba = model.predict_proba(X_val_selected)[:, 1]

        # Calculate AUC
        auc = roc_auc_score(y_val, y_val_proba)

        # Edge case 3: Sanity check (AUC should be >= 0.5)
        if auc < 0.5:
            if verbose:
                print(f"  Warning: AUC < 0.5 ({auc:.4f}), clipping to 0.5")
            auc = 0.5

        return float(auc)  # Ensure native Python float

    except Exception as e:
        # Edge case 2: Training failed
        if verbose:
            print(f"  Error: XGBoost training failed - {str(e)}")
        return 0.5  # Return baseline fitness (random guess)

print("Fitness function implemented:")
print("  - Method: XGBoost wrapper with validation AUC")
print("  - Edge cases: Empty subset, training failure, AUC sanity check")
print("  - Return range: [0.0, 1.0]")
print()

# ============================================================================
# @ SECTION 5: Grey Wolf Optimizer Class
# ============================================================================

print("# Grey Wolf Optimizer Class Implementation")
print("-" * 80)

class GreyWolfOptimizer:
    """
    Binary Grey Wolf Optimizer for feature selection.

    Implementation: Pure GWO with random constraint enforcement
    Reference: Mirjalili et al. (2014), Emary et al. (2016)

    Algorithm:
    1. Initialize population of wolves (random positions)
    2. Evaluate fitness and identify alpha, beta, delta wolves
    3. Update positions based on social hierarchy
    4. Convert continuous positions to binary
    5. Enforce feature constraints (random)
    6. Repeat until max iterations

    Attributes:
        n_wolves (int): Population size
        max_iterations (int): Maximum optimization iterations
        n_features (int): Number of features in dataset
        min_features (int): Minimum features to select
        max_features (int): Maximum features to select
        fitness_func (callable): Fitness evaluation function
        random_state (int): Random seed for reproducibility
        verbose (bool): Enable detailed logging
    """

    def __init__(self, n_wolves, max_iterations, n_features,
                 min_features, max_features, fitness_func,
                 random_state=42, verbose=True):
        """
        Initialize Grey Wolf Optimizer.

        Args:
            n_wolves (int): Population size (recommended: 30-50)
            max_iterations (int): Maximum iterations (recommended: 100-200)
            n_features (int): Total number of features
            min_features (int): Minimum features constraint
            max_features (int): Maximum features constraint
            fitness_func (callable): Fitness evaluation function
            random_state (int): Random seed
            verbose (bool): Enable logging
        """
        self.n_wolves = n_wolves
        self.max_iterations = max_iterations
        self.n_features = n_features
        self.min_features = min_features
        self.max_features = max_features
        self.fitness_func = fitness_func
        self.random_state = random_state
        self.verbose = verbose

        # Set random seed
        np.random.seed(random_state)

        # Initialize tracking
        self.convergence_history = []
        self.position_history = []

    def initialize_population(self):
        """
        Initialize wolf population with random continuous positions.

        Returns:
            np.ndarray: Population matrix (n_wolves x n_features)
        """
        population = np.random.rand(self.n_wolves, self.n_features)
        return population

    def update_positions(self, population, alpha_pos, beta_pos, delta_pos, a):
        """
        Update wolf positions based on alpha, beta, delta guidance.

        GWO Update Equations:
        D_alpha = |C1 * X_alpha - X_i|
        D_beta = |C2 * X_beta - X_i|
        D_delta = |C3 * X_delta - X_i|

        X1 = X_alpha - A1 * D_alpha
        X2 = X_beta - A2 * D_beta
        X3 = X_delta - A3 * D_delta

        X_i(t+1) = (X1 + X2 + X3) / 3

        Args:
            population (np.ndarray): Current population
            alpha_pos (np.ndarray): Alpha wolf position
            beta_pos (np.ndarray): Beta wolf position
            delta_pos (np.ndarray): Delta wolf position
            a (float): Exploration-exploitation parameter

        Returns:
            np.ndarray: Updated population
        """
        updated_population = np.zeros_like(population)

        for i in range(self.n_wolves):
            # Random coefficients
            r1 = np.random.rand(self.n_features)
            r2 = np.random.rand(self.n_features)

            A1 = 2 * a * r1 - a
            C1 = 2 * r2

            D_alpha = np.abs(C1 * alpha_pos - population[i])
            X1 = alpha_pos - A1 * D_alpha

            r1 = np.random.rand(self.n_features)
            r2 = np.random.rand(self.n_features)

            A2 = 2 * a * r1 - a
            C2 = 2 * r2

            D_beta = np.abs(C2 * beta_pos - population[i])
            X2 = beta_pos - A2 * D_beta

            r1 = np.random.rand(self.n_features)
            r2 = np.random.rand(self.n_features)

            A3 = 2 * a * r1 - a
            C3 = 2 * r2

            D_delta = np.abs(C3 * delta_pos - population[i])
            X3 = delta_pos - A3 * D_delta

            # Average of three positions
            updated_population[i] = (X1 + X2 + X3) / 3.0

            # Clip to [0, 1] range
            updated_population[i] = np.clip(updated_population[i], 0, 1)

        return updated_population

    def optimize(self, callback=None):
        """
        Execute Grey Wolf Optimization algorithm.

        Process:
        1. Initialize population
        2. For each iteration:
           a. Evaluate fitness for all wolves
           b. Identify alpha, beta, delta
           c. Update parameter 'a'
           d. Update wolf positions
           e. Convert to binary and enforce constraints
           f. Track convergence
           g. Call callback if provided

        Args:
            callback (callable, optional): Function to call after each iteration.
                                           Expected signature: callback(iteration, best_fitness, best_position).

        Returns:
            dict: Optimization results
        """
        if self.verbose:
            print(f"Starting GWO Optimization")
            print(f"  Population: {self.n_wolves} wolves")
            print(f"  Iterations: {self.max_iterations}")
            print(f"  Features: {self.n_features}")
            print(f"  Constraint: [{self.min_features}, {self.max_features}]")
            print()

        # Initialize population
        population = self.initialize_population()

        # Initialize best solutions
        alpha_pos = np.zeros(self.n_features)
        alpha_fitness = 0.0
        beta_pos = np.zeros(self.n_features)
        beta_fitness = 0.0
        delta_pos = np.zeros(self.n_features)
        delta_fitness = 0.0

        # Main optimization loop
        for iteration in range(self.max_iterations):
            # Calculate 'a' parameter (linearly decreases from 2 to 0)
            a = 2.0 - 2.0 * (iteration + 1) / self.max_iterations # Fix: Use iteration + 1

            # Evaluate fitness for all wolves
            fitness_values = []
            binary_population = []

            for i in range(self.n_wolves):
                # Convert to binary
                binary_pos = binary_conversion(population[i])

                # Enforce constraints (random)
                binary_pos = enforce_constraints(
                    binary_pos,
                    self.min_features,
                    self.max_features,
                    random_state=self.random_state + iteration + i
                )

                # Evaluate fitness
                fitness = self.fitness_func(binary_pos)

                fitness_values.append(fitness)
                binary_population.append(binary_pos)

            fitness_values = np.array(fitness_values)

            # Sort and identify alpha, beta, delta
            sorted_indices = np.argsort(fitness_values)[::-1]  # Descending order

            if fitness_values[sorted_indices[0]] > alpha_fitness:
                alpha_fitness = fitness_values[sorted_indices[0]]
                alpha_pos = population[sorted_indices[0]].copy()

            if fitness_values[sorted_indices[1]] > beta_fitness:
                beta_fitness = fitness_values[sorted_indices[1]]
                beta_pos = population[sorted_indices[1]].copy()

            if fitness_values[sorted_indices[2]] > delta_fitness:
                delta_fitness = fitness_values[sorted_indices[2]]
                delta_pos = population[sorted_indices[2]].copy()

            # Track convergence
            self.convergence_history.append(float(alpha_fitness))
            self.position_history.append(alpha_pos.copy())

            # Call callback if provided
            if callback:
                 # Ensure alpha_pos is converted to binary and constraints enforced for callback
                current_best_binary_pos = binary_conversion(alpha_pos)
                current_best_binary_pos = enforce_constraints(
                    current_best_binary_pos,
                    self.min_features,
                    self.max_features,
                    random_state=self.random_state + iteration # Use iteration for consistency
                )
                callback(iteration + 1, alpha_fitness, current_best_binary_pos) # Fix: Pass iteration + 1

            # Update positions
            population = self.update_positions(
                population, alpha_pos, beta_pos, delta_pos, a
            )

        # Final results
        best_binary_position = binary_conversion(alpha_pos)
        best_binary_position = enforce_constraints(
            best_binary_position,
            self.min_features,
            self.max_features,
            random_state=self.random_state # Use the initial random state for final conversion
        )

        # Calculate convergence iteration (95% of final fitness)
        convergence_threshold = 0.95 * alpha_fitness
        convergence_iteration = None

        for i, fitness in enumerate(self.convergence_history):
            if fitness >= convergence_threshold:
                convergence_iteration = i + 1
                break

        if convergence_iteration is None:
            convergence_iteration = self.max_iterations

        # Sanity check
        sanity_check = validate_threshold_effectiveness(self.position_history)

        if self.verbose:
            print()
            print(f"Optimization Complete")
            print(f"  Best Fitness: {alpha_fitness:.4f}")
            print(f"  Features Selected: {int(np.sum(best_binary_position))}")
            print(f"  Convergence Iteration: {convergence_iteration}")
            print(f"  Sanity Check Status: {sanity_check['status']}")
            print()

        return {
            'best_position': best_binary_position,
            'best_fitness': float(alpha_fitness),
            'convergence_curve': self.convergence_history,
            'convergence_iteration': int(convergence_iteration),
            'n_features_selected': int(np.sum(best_binary_position)),
            'total_iterations': int(self.max_iterations),
            'sanity_check': sanity_check
        }

print("GreyWolfOptimizer class implemented:")
print("  - Population-based optimization")
print("  - Social hierarchy: alpha, beta, delta, omega")
print("  - Random constraint enforcement (pure GWO)")
print("  - Convergence tracking and sanity checks")
print("  - Bug Fix: Type safety for all counting operations")
print("  - Enhancement: Added optional 'callback' parameter to optimize() method")
print()

# ============================================================================
# @ SECTION 6: Validation Tests
# ============================================================================

print("# Validation Tests")
print("-" * 80)

print("Test 1: Binary Conversion")
test_continuous = np.array([0.2, 0.5, 0.7, 0.49, 0.51, 0.0, 1.0])
test_binary = binary_conversion(test_continuous)
expected_binary = np.array([0, 1, 1, 0, 1, 0, 1])
assert np.array_equal(test_binary, expected_binary), "Binary conversion test failed"
print(f"  Input: {test_continuous}")
print(f"  Output: {test_binary}")
print(f"  Expected: {expected_binary}")
print("  Status: PASSED")
print()

print("Test 2: Constraint Enforcement - Too Few Features")
test_binary_few = np.array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0] + [0]*15)  # Only 2 features
result_few = enforce_constraints(test_binary_few, min_features=8, max_features=15, random_state=42)
n_selected_few = int(np.sum(result_few))
assert n_selected_few >= 8, "Constraint enforcement (too few) failed"
print(f"  Input features: {int(np.sum(test_binary_few))}")
print(f"  Output features: {n_selected_few}")
print(f"  Status: PASSED (enforced minimum)")
print()

print("Test 3: Constraint Enforcement - Too Many Features")
test_binary_many = np.ones(25)  # All 25 features
result_many = enforce_constraints(test_binary_many, min_features=8, max_features=15, random_state=42)
n_selected_many = int(np.sum(result_many))
assert n_selected_many <= 15, "Constraint enforcement (too many) failed"
print(f"  Input features: {int(np.sum(test_binary_many))}")
print(f"  Output features: {n_selected_many}")
print(f"  Status: PASSED (enforced maximum)")
print()

print("Test 4: Constraint Enforcement - Within Range")
test_binary_ok = np.array([1]*10 + [0]*15)  # 10 features (within [8,15])
result_ok = enforce_constraints(test_binary_ok, min_features=8, max_features=15, random_state=42)
assert np.array_equal(result_ok, test_binary_ok), "Constraint enforcement (within range) should not modify"
print(f"  Input features: {int(np.sum(test_binary_ok))}")
print(f"  Output features: {int(np.sum(result_ok))}")
print(f"  Status: PASSED (no modification needed)")
print()

print("Test 5: Fitness Function - Small Scale Test")
# Create small test fitness function
def test_fitness_wrapper(feature_subset):
    return fitness_function(
        feature_subset,
        X_train.head(100),
        y_train.head(100),
        X_val.head(50),
        y_val.head(50),
        feature_names,
        XGBOOST_FITNESS_CONFIG,
        verbose=False
    )

test_subset = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0] + [0]*15)  # First 5 features
test_fitness = test_fitness_wrapper(test_subset)
assert 0.0 <= test_fitness <= 1.0, "Fitness should be in [0,1] range"
print(f"  Test subset: {int(np.sum(test_subset))} features selected")
print(f"  Fitness (AUC): {test_fitness:.4f}")
print(f"  Status: PASSED (valid AUC range)")
print()

print("Test 6: GWO Initialization and Single Iteration")
# Test GWO with minimal configuration
test_gwo = GreyWolfOptimizer(
    n_wolves=5,
    max_iterations=2,
    n_features=25,
    min_features=8,
    max_features=15,
    fitness_func=test_fitness_wrapper,
    random_state=42,
    verbose=False
)

test_population = test_gwo.initialize_population()
assert test_population.shape == (5, 25), "Population shape mismatch"
assert np.all((test_population >= 0) & (test_population <= 1)), "Population values out of range"
print(f"  Population shape: {test_population.shape}")
print(f"  Value range: [{test_population.min():.3f}, {test_population.max():.3f}]")
print(f"  Status: PASSED (valid initialization)")
print()

print("Test 7: GWO Optimization with Callback")
# Test GWO with a callback
callback_calls = []
def simple_callback(iteration, best_fitness, best_position):
    callback_calls.append((iteration, best_fitness, best_position.copy()))

test_gwo_callback = GreyWolfOptimizer(
    n_wolves=5,
    max_iterations=5,
    n_features=25,
    min_features=8,
    max_features=15,
    fitness_func=test_fitness_wrapper,
    random_state=42,
    verbose=False
)

test_gwo_callback.optimize(callback=simple_callback)

assert len(callback_calls) == 5, f"Expected 5 callback calls, got {len(callback_calls)}"
print(f"  Callback test: Expected 5 calls, got {len(callback_calls)}")
print("  Status: PASSED (callback invoked)")
print()


print("All validation tests: PASSED")
print()

# ============================================================================
# @ SECTION 7: Save Configurations and Models
# ============================================================================

print("# Saving Configurations")
print("-" * 80)

# GWO configuration
gwo_configuration = {
    'algorithm': 'Grey Wolf Optimizer (GWO)',
    'implementation': 'Pure GWO with Random Constraint Enforcement',
    'reference': 'Mirjalili et al. (2014), Emary et al. (2016)',
    'bug_fixes': [
        {
            'issue': 'TypeError in np.random.choice',
            'cause': 'numpy scalar types (np.int64, np.float64) not accepted',
            'solution': 'Explicit type conversion to Python native int',
            'affected_functions': ['enforce_constraints', 'fitness_function', 'optimize']
        },
         {
            'issue': 'TypeError: optimize() got an unexpected keyword argument "callback"',
            'cause': 'optimize method did not accept a callback parameter',
            'solution': 'Added optional callback parameter to optimize() and invoked it within the loop',
            'affected_functions': ['optimize']
        },
         {
            'issue': 'Incorrect "a" parameter calculation',
            'cause': 'Calculation was `2.0 - 2.0 * iteration / self.max_iterations` which resulted in `a` being 2.0 at the last iteration instead of 0.0',
            'solution': 'Changed calculation to `2.0 - 2.0 * (iteration + 1) / self.max_iterations`',
            'affected_functions': ['optimize']
        }
    ],
    'parameters': {
        'min_features': GWO_CONFIG['min_features'],
        'max_features': GWO_CONFIG['max_features'],
        'random_state': GWO_CONFIG['random_state']
    },
    'binary_conversion': {
        'method': 'simple_threshold',
        'threshold': 0.5,
        'reference': 'Emary et al. (2016)'
    },
    'constraint_enforcement': {
        'strategy': 'random',
        'description': 'Unbiased random add/remove to satisfy [min, max] constraints',
        'rationale': 'Preserves GWO stochastic exploration, aligned with literature'
    },
    'fitness_function': {
        'method': 'XGBoost wrapper',
        'metric': 'validation_auc',
        'xgboost_config': XGBOOST_FITNESS_CONFIG
    },
    'sanity_checks': {
        'clarity_score': {
            'excellent': '>0.35',
            'good': '0.25-0.35',
            'acceptable': '0.15-0.25',
            'problematic': '<0.15'
        },
        'oscillation_pct': {
            'excellent': '<0.10',
            'good': '0.10-0.20',
            'acceptable': '0.20-0.30',
            'problematic': '>0.30'
        }
    },
    'metadata': {
        'cell': 'Cell 2',
        'notebook': '03_Proposed_GWO_XGBoost.ipynb',
        'author': 'Chatelia Dyah Prameswari (1301223320)',
        'timestamp': datetime.now().isoformat(),
        'version': '1.2_bugfix_callback' # Updated version
    }
}

# Save configuration
config_path = CONFIG_DIR / 'gwo_configuration.json'
with open(config_path, 'w') as f:
    json.dump(gwo_configuration, f, indent=2)

print(f"GWO configuration saved: {config_path.name}")

# Save validation test results
validation_results = {
    'test_summary': {
        'total_tests': 7, # Updated total tests
        'passed': 7, # Updated passed tests
        'failed': 0,
        'status': 'all_passed'
    },
    'tests': [
        {'test_id': 1, 'name': 'Binary Conversion', 'status': 'passed'},
        {'test_id': 2, 'name': 'Constraint Enforcement - Too Few', 'status': 'passed'},
        {'test_id': 3, 'name': 'Constraint Enforcement - Too Many', 'status': 'passed'},
        {'test_id': 4, 'name': 'Constraint Enforcement - Within Range', 'status': 'passed'},
        {'test_id': 5, 'name': 'Fitness Function', 'status': 'passed'},
        {'test_id': 6, 'name': 'GWO Initialization', 'status': 'passed'},
        {'test_id': 7, 'name': 'GWO Optimization with Callback', 'status': 'passed'} # Added new test
    ],
    'timestamp': datetime.now().isoformat()
}

validation_path = TEST_DIR / 'validation_results.json'
with open(validation_path, 'w') as f:
    json.dump(validation_results, f, indent=2)

print(f"Validation results saved: {validation_path.name}")

# Save GWO class metadata
gwo_class_metadata = {
    'class_name': 'GreyWolfOptimizer',
    'methods': [
        'initialize_population',
        'update_positions',
        'optimize'
    ],
    'attributes': [
        'n_wolves',
        'max_iterations',
        'n_features',
        'min_features',
        'max_features',
        'fitness_func',
        'random_state',
        'verbose'
    ],
    'return_structure': {
        'best_position': 'Binary feature vector (np.ndarray)',
        'best_fitness': 'Validation AUC (float)',
        'convergence_curve': 'Fitness per iteration (list)',
        'convergence_iteration': 'Iteration reaching 95% final fitness (int)',
        'n_features_selected': 'Number of selected features (int)',
        'total_iterations': 'Maximum iterations executed (int)',
        'sanity_check': 'Post-optimization validation metrics (dict)'
    },
    'usage_example': {
        'initialization': 'gwo = GreyWolfOptimizer(n_wolves=40, max_iterations=150, ...)',
        'optimization': 'results = gwo.optimize()',
        'access_results': 'best_features = results["best_position"]'
    }
}

metadata_path = MODEL_DIR / 'gwo_class_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(gwo_class_metadata, f, indent=2)

print(f"GWO class metadata saved: {metadata_path.name}")
print()

# ============================================================================
# @ SECTION 8: Summary
# ============================================================================

print("=" * 80)
print("CELL 2 EXECUTION SUMMARY")
print("=" * 80)
print()
print("Status: COMPLETED SUCCESSFULLY")
print()
print("Implementation Summary:")
print("  - Binary Conversion: Simple threshold (>=0.5)")
print("  - Constraint Enforcement: Pure Random (unbiased)")
print("  - Fitness Function: XGBoost + Validation AUC")
print("  - GWO Class: Complete with convergence tracking and callback support")
print("  - Sanity Checks: Clarity score + Oscillation analysis")
print("  - Validation Tests: 7/7 passed") # Updated summary
print("  - Bug Fixes: Type safety for numpy scalar operations, callback support, 'a' parameter calculation") # Updated summary
print()
print("Output Files:")
print(f"  - Configuration: {config_path}")
print(f"  - Validation Results: {validation_path}")
print(f"  - Class Metadata: {metadata_path}")
print()
print("Ready for Cell 3: Coarse Grid Search")
print("=" * 80)

CELL 2: GREY WOLF OPTIMIZER CORE IMPLEMENTATION
Implementation: Pure GWO with Random Constraint Enforcement

# Output Directory Configuration
--------------------------------------------------------------------------------
Output Directory: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_2_pureGWO
  - Configurations: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_2_pureGWO/configurations
  - Validation Tests: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_2_pureGWO/validation_tests
  - GWO Models: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_2_pureGWO/gwo_models

# Binary Conversion Implementation
--------------------------------------------------------------------------------
Binary conversion utilities implemented:
  - binary_conversion(): Simple threshold (>=0.5)
  - validate_threshold_effectiveness(): Post-optimization sanity check

# Constrai

In [ ]:
# @title Cell 5: Final Experiments - Training Phase (10 Runs)
"""
================================================================================
CELL 5: FINAL EXPERIMENTS - TRAINING PHASE
================================================================================
Objectives:
1. Execute 10 independent GWO runs with optimal parameters
2. Train final XGBoost models on selected features
3. Evaluate on VALIDATION SET only (NO test set evaluation)
4. Save all trained models and training metadata
5. Per-run checkpointing for crash recovery

CRITICAL: Test set is NOT evaluated in this cell. Testing is in Cell 6.

Expected Runtime: ~2 hours (GPU L4) or ~3-4 hours (CPU fallback)
Output Directory: .../cell_5_final_run_training/
================================================================================
"""

import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score,
    recall_score, f1_score, log_loss
)
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PART 1: GPU AVAILABILITY CHECK AND CONFIGURATION
# ============================================================================

print("="*80)
print("GPU AVAILABILITY CHECK")
print("="*80)
print()

def check_gpu_availability():
    """
    Check if GPU is available for XGBoost.

    Returns:
        tuple: (is_available: bool, gpu_name: str, config: dict)
    """
    try:
        import subprocess
        gpu_info = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
            stderr=subprocess.DEVNULL
        )
        gpu_name = gpu_info.decode('utf-8').strip().split(',')[0]

        # Test XGBoost GPU
        import xgboost as xgb
        test_model = xgb.XGBClassifier(
            tree_method='gpu_hist',
            predictor='gpu_predictor',
            n_estimators=10
        )

        # Small test
        X_test = np.random.rand(100, 10)
        y_test = np.random.randint(0, 2, 100)
        test_model.fit(X_test, y_test, verbose=False)

        gpu_config = {
            'tree_method': 'gpu_hist',
            'predictor': 'gpu_predictor'
        }

        return True, gpu_name, gpu_config

    except Exception as e:
        return False, None, {
            'tree_method': 'hist',
            'predictor': 'cpu_predictor'
        }

gpu_available, gpu_name, compute_config = check_gpu_availability()

if gpu_available:
    print(f"GPU DETECTED: {gpu_name}")
    print("  - Using GPU acceleration for XGBoost")
    print(f"  - tree_method: {compute_config['tree_method']}")
    print(f"  - predictor: {compute_config['predictor']}")
else:
    print("GPU NOT AVAILABLE")
    print("  - Falling back to CPU computation")
    print(f"  - tree_method: {compute_config['tree_method']}")
    print(f"  - predictor: {compute_config['predictor']}")
    print()
    print("WARNING: Runtime will be significantly longer (~2-3x slower)")

print()

# ============================================================================
# PART 2: CONFIGURATION AND SETUP
# ============================================================================

print("="*80)
print("TRAINING PHASE INITIALIZATION")
print("="*80)
print()

# Define output directory
BASE_DIR = Path('/content/drive/MyDrive/ASD_GWO_XGBoost_Project')
OUTPUT_DIR = BASE_DIR / '03_Results/output_notebook_03/cell_5_final_run_training'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output Directory: {OUTPUT_DIR}")
print()

# Load optimal parameters from Cell 4
optimal_params_file = BASE_DIR / '03_Results/output_notebook_03/cell_4_fine_grid/optimal_parameters.json'

print("Loading optimal parameters from Cell 4...")
with open(optimal_params_file, 'r') as f:
    optimal_params = json.load(f)

population_size = optimal_params['population_size']
max_iterations = optimal_params['max_iterations']

print(f"  Population Size: {population_size}")
print(f"  Max Iterations: {max_iterations}")
print()

# Define experiment configuration
RANDOM_SEEDS = list(range(10))
TOTAL_RUNS = len(RANDOM_SEEDS)

print(f"Experiment Configuration:")
print(f"  Total Runs: {TOTAL_RUNS}")
print(f"  Random Seeds: {RANDOM_SEEDS}")
print()

# XGBoost Configuration - DYNAMIC BASED ON GPU AVAILABILITY
xgb_config = {
    'n_estimators': 500,
    'max_depth': 8,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': compute_config['tree_method'],  # Dynamic
    'predictor': compute_config['predictor'],      # Dynamic
    'eval_metric': 'auc',
    'random_state': 42,
    'objective': 'binary:logistic',
    'verbosity': 0
}

# Fitness XGBoost Configuration - DYNAMIC
fitness_xgb_config = {
    'n_estimators': 150,
    'max_depth': 6,
    'learning_rate': 0.1,
    'tree_method': compute_config['tree_method'],  # Dynamic
    'predictor': compute_config['predictor'],      # Dynamic
    'random_state': 42,
    'verbosity': 0
}

print("XGBoost Configuration (Final Model):")
for key, value in xgb_config.items():
    print(f"  {key}: {value}")
print()

# Load preprocessed datasets
print("Loading preprocessed datasets...")
PREPROCESSED_DIR = BASE_DIR / '01_Dataset/splits/no_ethnicity/preprocessed'

train_data = pd.read_csv(PREPROCESSED_DIR / 'train_set_preprocessed.csv')
val_data = pd.read_csv(PREPROCESSED_DIR / 'val_set_preprocessed.csv')

TARGET_COL = 'ASD_traits'

if TARGET_COL not in train_data.columns:
    raise ValueError(f"Target column '{TARGET_COL}' not found in dataset.")

X_train = train_data.drop(columns=[TARGET_COL])
y_train = train_data[TARGET_COL].values

X_val = val_data.drop(columns=[TARGET_COL])
y_val = val_data[TARGET_COL].values

print(f"  Training Set: {X_train.shape} (samples, features)")
print(f"  Validation Set: {X_val.shape}")
print(f"  Target Distribution - Train: {dict(zip(*np.unique(y_train, return_counts=True)))}")
print()

# Feature constraints
MIN_FEATURES = 8
MAX_FEATURES = 15
N_FEATURES = X_train.shape[1]
FEATURE_NAMES = X_train.columns.tolist()

print(f"Feature Configuration:")
print(f"  Total Features: {N_FEATURES}")
print(f"  Feature Constraints: {MIN_FEATURES}-{MAX_FEATURES}")
print()

# ============================================================================
# PART 3: GREY WOLF OPTIMIZER IMPLEMENTATION
# ============================================================================

class GreyWolfOptimizer:
    """Grey Wolf Optimizer for Feature Selection"""

    def __init__(self, n_wolves, max_iter, n_features, min_features, max_features,
                 fitness_func, random_seed):
        self.n_wolves = n_wolves
        self.max_iter = max_iter
        self.n_features = n_features
        self.min_features = min_features
        self.max_features = max_features
        self.fitness_func = fitness_func
        self.random_seed = random_seed
        np.random.seed(random_seed)

        self.alpha_pos = None
        self.alpha_score = float('-inf')
        self.convergence_curve = []

    def initialize_population(self):
        """Initialize wolf population with random binary positions"""
        population = []
        for _ in range(self.n_wolves):
            position = np.random.rand(self.n_features)
            binary_position = self._to_binary(position)
            binary_position = self._enforce_constraints(binary_position)
            population.append(binary_position)
        return np.array(population)

    def _to_binary(self, position):
        """Convert continuous position to binary"""
        return (position >= 0.5).astype(int)

    def _enforce_constraints(self, position):
        """Ensure feature count is within constraints"""
        n_selected = int(np.sum(position))

        if n_selected < self.min_features:
            zero_indices = np.where(position == 0)[0]
            add_count = self.min_features - n_selected
            add_indices = np.random.choice(zero_indices, size=add_count, replace=False)
            position[add_indices] = 1

        elif n_selected > self.max_features:
            one_indices = np.where(position == 1)[0]
            remove_count = n_selected - self.max_features
            remove_indices = np.random.choice(one_indices, size=remove_count, replace=False)
            position[remove_indices] = 0

        return position

    def optimize(self):
        """Main GWO optimization loop"""
        population = self.initialize_population()

        alpha_pos = population[0].copy()
        alpha_score = float('-inf')
        beta_pos = population[0].copy()
        beta_score = float('-inf')
        delta_pos = population[0].copy()
        delta_score = float('-inf')

        # Evaluate initial population
        for i, wolf in enumerate(population):
            fitness = self.fitness_func(wolf)

            if fitness > alpha_score:
                delta_score = beta_score
                delta_pos = beta_pos.copy()
                beta_score = alpha_score
                beta_pos = alpha_pos.copy()
                alpha_score = fitness
                alpha_pos = wolf.copy()
            elif fitness > beta_score:
                delta_score = beta_score
                delta_pos = beta_pos.copy()
                beta_score = fitness
                beta_pos = wolf.copy()
            elif fitness > delta_score:
                delta_score = fitness
                delta_pos = wolf.copy()

        self.convergence_curve.append(alpha_score)

        # Main optimization loop
        for iteration in range(self.max_iter):
            a = 2 - iteration * (2.0 / self.max_iter)

            for i in range(self.n_wolves):
                for j in range(self.n_features):
                    r1 = np.random.random()
                    r2 = np.random.random()

                    A1 = 2 * a * r1 - a
                    C1 = 2 * r2

                    D_alpha = abs(C1 * alpha_pos[j] - population[i][j])
                    X1 = alpha_pos[j] - A1 * D_alpha

                    r1 = np.random.random()
                    r2 = np.random.random()

                    A2 = 2 * a * r1 - a
                    C2 = 2 * r2

                    D_beta = abs(C2 * beta_pos[j] - population[i][j])
                    X2 = beta_pos[j] - A2 * D_beta

                    r1 = np.random.random()
                    r2 = np.random.random()

                    A3 = 2 * a * r1 - a
                    C3 = 2 * r2

                    D_delta = abs(C3 * delta_pos[j] - population[i][j])
                    X3 = delta_pos[j] - A3 * D_delta

                    population[i][j] = (X1 + X2 + X3) / 3.0

                population[i] = self._to_binary(population[i])
                population[i] = self._enforce_constraints(population[i])

                fitness = self.fitness_func(population[i])

                if fitness > alpha_score:
                    delta_score = beta_score
                    delta_pos = beta_pos.copy()
                    beta_score = alpha_score
                    beta_pos = alpha_pos.copy()
                    alpha_score = fitness
                    alpha_pos = population[i].copy()
                elif fitness > beta_score:
                    delta_score = beta_score
                    delta_pos = beta_pos.copy()
                    beta_score = fitness
                    beta_pos = population[i].copy()
                elif fitness > delta_score:
                    delta_score = fitness
                    delta_pos = population[i].copy()

            self.convergence_curve.append(alpha_score)

        self.alpha_pos = alpha_pos
        self.alpha_score = alpha_score

        return alpha_pos, alpha_score, self.convergence_curve

# ============================================================================
# PART 4: CHECKPOINT MANAGEMENT
# ============================================================================

def load_checkpoint():
    """Load checkpoint if exists"""
    checkpoint_file = OUTPUT_DIR / 'training_checkpoint.json'
    if checkpoint_file.exists():
        with open(checkpoint_file, 'r') as f:
            return json.load(f)
    return None

def save_checkpoint(checkpoint_data):
    """Save checkpoint incrementally"""
    checkpoint_file = OUTPUT_DIR / 'training_checkpoint.json'
    with open(checkpoint_file, 'w') as f:
        json.dump(checkpoint_data, f, indent=2)

def get_completed_runs(checkpoint):
    """Get list of completed run IDs"""
    if checkpoint is None:
        return []
    return [run['run_id'] for run in checkpoint.get('runs', [])]

# ============================================================================
# PART 5: EVALUATION FUNCTIONS
# ============================================================================

def calculate_metrics(y_true, y_pred, y_proba):
    """Calculate comprehensive classification metrics with macro averaging"""
    metrics = {
        'roc_auc': float(roc_auc_score(y_true, y_proba)),
        'accuracy': float(accuracy_score(y_true, y_pred)),
        'precision_macro': float(precision_score(y_true, y_pred, average='macro', zero_division=0)),
        'recall_macro': float(recall_score(y_true, y_pred, average='macro', zero_division=0)),
        'f1_macro': float(f1_score(y_true, y_pred, average='macro', zero_division=0)),
        'log_loss': float(log_loss(y_true, y_proba, labels=[1, 2]))
    }
    return metrics

def fitness_function(feature_mask):
    """Fitness function for GWO optimization - uses validation AUC"""
    selected_features = [FEATURE_NAMES[i] for i in range(len(feature_mask)) if feature_mask[i] == 1]

    if len(selected_features) == 0:
        return 0.0

    X_train_selected = X_train[selected_features]
    X_val_selected = X_val[selected_features]

    model = XGBClassifier(**fitness_xgb_config)
    model.fit(X_train_selected, y_train, verbose=False)

    y_val_proba = model.predict_proba(X_val_selected)[:, 1]
    auc = roc_auc_score(y_val, y_val_proba)

    return auc

# ============================================================================
# PART 6: MAIN TRAINING LOOP
# ============================================================================

print("="*80)
print("TRAINING PHASE EXECUTION")
print("="*80)
print()

# Load checkpoint if exists
checkpoint = load_checkpoint()
completed_runs = get_completed_runs(checkpoint)

if checkpoint is not None:
    print(f"Checkpoint found: {len(completed_runs)} runs already completed")
    print(f"Resuming from run {len(completed_runs)}")
    print()
    all_runs = checkpoint['runs']
else:
    print("No checkpoint found. Starting from scratch.")
    print()
    all_runs = []

# Create models directory
models_dir = OUTPUT_DIR / 'models'
models_dir.mkdir(exist_ok=True)

start_time_total = time.time()

for run_idx, seed in enumerate(RANDOM_SEEDS):
    run_id = run_idx

    if run_id in completed_runs:
        print(f"Run {run_id + 1}/{TOTAL_RUNS} already completed (seed={seed}). Skipping...")
        print()
        continue

    print("="*80)
    print(f"RUN {run_id + 1}/{TOTAL_RUNS} - SEED {seed}")
    print("="*80)
    print()

    run_start_time = time.time()

    # ========================================================================
    # STEP 1: GWO Feature Selection
    # ========================================================================
    print(f"Step 1: GWO Optimization (Pop={population_size}, Iter={max_iterations})")
    print()

    gwo = GreyWolfOptimizer(
        n_wolves=population_size,
        max_iter=max_iterations,
        n_features=N_FEATURES,
        min_features=MIN_FEATURES,
        max_features=MAX_FEATURES,
        fitness_func=fitness_function,
        random_seed=seed
    )

    best_position, best_fitness, convergence_history = gwo.optimize()

    selected_features = [FEATURE_NAMES[i] for i in range(len(best_position)) if best_position[i] == 1]
    n_features_selected = len(selected_features)

    convergence_threshold = 0.95 * best_fitness
    convergence_iteration = next(
        (i for i, fitness in enumerate(convergence_history) if fitness >= convergence_threshold),
        len(convergence_history) - 1
    )

    print(f"GWO Optimization Complete:")
    print(f"  Best Validation AUC (GWO): {best_fitness:.4f}")
    print(f"  Features Selected: {n_features_selected}")
    print(f"  Convergence Iteration: {convergence_iteration + 1}/{max_iterations}")
    print()

    # ========================================================================
    # STEP 2: Train Final Model on Selected Features
    # ========================================================================
    print("Step 2: Training Final Model on Selected Features")
    print()

    X_train_selected = X_train[selected_features]
    X_val_selected = X_val[selected_features]

    # Create run-specific XGBoost config with unique random_state
    run_xgb_config = xgb_config.copy()
    run_xgb_config['random_state'] = seed

    final_model = XGBClassifier(**run_xgb_config)
    final_model.fit(
        X_train_selected,
        y_train,
        eval_set=[(X_val_selected, y_val)],
        verbose=False
    )

    print("Model training complete.")
    print()

    # ========================================================================
    # STEP 3: Validation Set Evaluation ONLY
    # ========================================================================
    print("Step 3: VALIDATION SET Evaluation")
    print()

    y_val_pred = final_model.predict(X_val_selected)
    y_val_proba = final_model.predict_proba(X_val_selected)[:, 1]
    validation_metrics = calculate_metrics(y_val, y_val_pred, y_val_proba)

    print("Validation Metrics (Macro Averaging):")
    print(f"  AUC: {validation_metrics['roc_auc']:.4f}")
    print(f"  Accuracy: {validation_metrics['accuracy']:.4f}")
    print(f"  Precision (macro): {validation_metrics['precision_macro']:.4f}")
    print(f"  Recall (macro): {validation_metrics['recall_macro']:.4f}")
    print(f"  F1-Score (macro): {validation_metrics['f1_macro']:.4f}")
    print(f"  Log Loss: {validation_metrics['log_loss']:.4f}")
    print()

    # ========================================================================
    # STEP 4: Save Model
    # ========================================================================
    model_path = models_dir / f'run_{run_id}_model.json'
    final_model.save_model(str(model_path))
    print(f"Model saved to: {model_path}")
    print()

    # ========================================================================
    # STEP 5: Save Run Results
    # ========================================================================
    run_duration = time.time() - run_start_time

    run_result = {
        'run_id': run_id,
        'random_seed': seed,
        'selected_features': selected_features,
        'n_features': n_features_selected,
        'validation_auc_gwo': float(best_fitness),
        'validation_metrics': validation_metrics,
        'convergence_iteration': convergence_iteration,
        'convergence_history': [float(x) for x in convergence_history],
        'model_path': str(model_path),
        'runtime_seconds': float(run_duration)
    }

    all_runs.append(run_result)

    # Save checkpoint
    checkpoint_data = {
        'checkpoint_metadata': {
            'last_completed_run': run_id,
            'timestamp': datetime.now().isoformat(),
            'total_runs': TOTAL_RUNS,
            'gpu_available': gpu_available
        },
        'optimal_parameters': {
            'population_size': population_size,
            'max_iterations': max_iterations
        },
        'compute_config': compute_config,
        'runs': all_runs
    }
    save_checkpoint(checkpoint_data)

    print(f"Run {run_id + 1}/{TOTAL_RUNS} Complete")
    print(f"  Runtime: {run_duration/60:.2f} minutes")
    print()

    # Progress tracking
    elapsed_time = time.time() - start_time_total
    avg_time_per_run = elapsed_time / (run_id + 1)
    remaining_runs = TOTAL_RUNS - (run_id + 1)
    eta_seconds = avg_time_per_run * remaining_runs

    print(f"Progress: {((run_id + 1) / TOTAL_RUNS * 100):.1f}% ({run_id + 1}/{TOTAL_RUNS})")
    print(f"Elapsed: {elapsed_time/3600:.2f} hours | ETA: {eta_seconds/3600:.2f} hours")
    print()

total_duration = time.time() - start_time_total

print("="*80)
print("ALL TRAINING RUNS COMPLETED")
print("="*80)
print(f"Total Training Time: {total_duration/3600:.2f} hours")
print()

# ============================================================================
# PART 7: AGGREGATE VALIDATION RESULTS
# ============================================================================

print("="*80)
print("TRAINING RESULTS SUMMARY")
print("="*80)
print()

val_aucs = [run['validation_metrics']['roc_auc'] for run in all_runs]
val_f1_scores = [run['validation_metrics']['f1_macro'] for run in all_runs]
n_features_list = [run['n_features'] for run in all_runs]

statistics = {
    'validation_auc_mean': float(np.mean(val_aucs)),
    'validation_auc_std': float(np.std(val_aucs)),
    'validation_auc_ci_lower': float(np.percentile(val_aucs, 2.5)),
    'validation_auc_ci_upper': float(np.percentile(val_aucs, 97.5)),
    'validation_f1_macro_mean': float(np.mean(val_f1_scores)),
    'validation_f1_macro_std': float(np.std(val_f1_scores)),
    'n_features_mean': float(np.mean(n_features_list)),
    'n_features_std': float(np.std(n_features_list)),
    'n_features_min': int(np.min(n_features_list)),
    'n_features_max': int(np.max(n_features_list)),
    'best_run_index': int(np.argmax(val_aucs))
}

print("Validation Performance Summary:")
print(f"  AUC: {statistics['validation_auc_mean']:.4f} +/- {statistics['validation_auc_std']:.4f}")
print(f"       95% CI: [{statistics['validation_auc_ci_lower']:.4f}, {statistics['validation_auc_ci_upper']:.4f}]")
print(f"  F1-Score (macro): {statistics['validation_f1_macro_mean']:.4f} +/- {statistics['validation_f1_macro_std']:.4f}")
print(f"  Features: {statistics['n_features_mean']:.1f} +/- {statistics['n_features_std']:.1f}")
print(f"            Range: [{statistics['n_features_min']}, {statistics['n_features_max']}]")
print(f"  Best Run: {statistics['best_run_index']} (Val AUC: {val_aucs[statistics['best_run_index']]:.4f})")
print()

# ============================================================================
# PART 8: SAVE TRAINING RESULTS
# ============================================================================

print("="*80)
print("SAVING TRAINING RESULTS")
print("="*80)
print()

training_results = {
    'experiment_info': {
        'name': 'GWO-XGBoost Training Phase',
        'description': '10 independent runs with optimal GWO parameters',
        'timestamp': datetime.now().isoformat(),
        'n_runs': TOTAL_RUNS,
        'total_runtime_hours': total_duration / 3600,
        'gpu_available': gpu_available,
        'gpu_name': gpu_name if gpu_available else 'CPU'
    },
    'optimal_parameters': {
        'population_size': population_size,
        'max_iterations': max_iterations,
        'min_features': MIN_FEATURES,
        'max_features': MAX_FEATURES
    },
    'xgboost_configuration': xgb_config,
    'compute_config': compute_config,
    'runs': all_runs,
    'statistics': statistics
}

results_file = OUTPUT_DIR / 'gwo_training_results.json'
with open(results_file, 'w') as f:
    json.dump(training_results, f, indent=2)

print(f"Training results saved to: {results_file}")
print()

# Cleanup checkpoint
checkpoint_file = OUTPUT_DIR / 'training_checkpoint.json'
if checkpoint_file.exists():
    checkpoint_file.unlink()
    print("Training checkpoint cleaned up.")
print()

print("="*80)
print("TRAINING PHASE COMPLETED SUCCESSFULLY")
print("="*80)
print()

print("Summary:")
print(f"  Total Runs: {TOTAL_RUNS}")
print(f"  Models Saved: {len(all_runs)}")
print(f"  Validation AUC: {statistics['validation_auc_mean']:.4f} +/- {statistics['validation_auc_std']:.4f}")
print(f"  Average Features: {statistics['n_features_mean']:.1f}")
print(f"  Compute Mode: {'GPU (' + gpu_name + ')' if gpu_available else 'CPU'}")
print()

print("Output Files:")
print(f"  1. Training Results: {results_file}")
print(f"  2. Trained Models: {models_dir} (10 models)")
print()

print("CRITICAL NOTE: Test set NOT evaluated in this cell.")
print("Proceed to Cell 6 for unbiased test set evaluation.")
print("="*80)

GPU AVAILABILITY CHECK

GPU NOT AVAILABLE
  - Falling back to CPU computation
  - tree_method: hist
  - predictor: cpu_predictor


TRAINING PHASE INITIALIZATION

Output Directory: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_5_final_run_training

Loading optimal parameters from Cell 4...
  Population Size: 25
  Max Iterations: 90

Experiment Configuration:
  Total Runs: 10
  Random Seeds: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

XGBoost Configuration (Final Model):
  n_estimators: 500
  max_depth: 8
  learning_rate: 0.05
  subsample: 0.8
  colsample_bytree: 0.8
  tree_method: hist
  predictor: cpu_predictor
  eval_metric: auc
  random_state: 42
  objective: binary:logistic
  verbosity: 0

Loading preprocessed datasets...
  Training Set: (1270, 25) (samples, features)
  Validation Set: (318, 25)
  Target Distribution - Train: {np.int64(0): np.int64(583), np.int64(1): np.int64(687)}

Feature Configuration:
  Total Features: 25
  Feature Constraints: 8-15

TRAI

In [ ]:
# @title Cell 6: Final Experiments - Testing Phase (10 Models)

"""
================================================================================
CELL 6: FINAL EXPERIMENTS - TESTING PHASE
================================================================================
Objectives:
1. Load all 10 trained models from Cell 5
2. Evaluate each model on held-out test set
3. Calculate comprehensive test metrics with macro averaging
4. Generate statistical summary and confidence intervals
5. Identify best performing model

CRITICAL: Unbiased evaluation on test set that was never seen during training.

Expected Runtime: ~15 minutes
Output Directory: .../cell_6_final_run_testing/
================================================================================
"""

import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score,
    recall_score, f1_score, log_loss, confusion_matrix,
    roc_curve
)
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PART 1: CONFIGURATION AND SETUP
# ============================================================================

print("="*80)
print("TESTING PHASE INITIALIZATION")
print("="*80)
print()

# Define directories
BASE_DIR = Path('/content/drive/MyDrive/ASD_GWO_XGBoost_Project')
TRAINING_DIR = BASE_DIR / '03_Results/output_notebook_03/cell_5_final_run_training'
OUTPUT_DIR = BASE_DIR / '03_Results/output_notebook_03/cell_6_final_run_testing'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Training Directory: {TRAINING_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")
print()

# Load training results
training_results_file = TRAINING_DIR / 'gwo_training_results.json'

if not training_results_file.exists():
    raise FileNotFoundError(f"Training results not found: {training_results_file}")

print("Loading training results from Cell 5...")
with open(training_results_file, 'r') as f:
    training_results = json.load(f)

n_runs = training_results['experiment_info']['n_runs']
print(f"  Number of trained models: {n_runs}")
print()

# Load test set
print("Loading test set...")
PREPROCESSED_DIR = BASE_DIR / '01_Dataset/splits/no_ethnicity/preprocessed'
test_data = pd.read_csv(PREPROCESSED_DIR / 'test_set_preprocessed.csv')

TARGET_COL = 'ASD_traits'

if TARGET_COL not in test_data.columns:
    raise ValueError(f"Target column '{TARGET_COL}' not found in test set.")

X_test = test_data.drop(columns=[TARGET_COL])
y_test = test_data[TARGET_COL].values

print(f"  Test Set: {X_test.shape} (samples, features)")
print(f"  Target Distribution: {dict(zip(*np.unique(y_test, return_counts=True)))}")
print()

# ============================================================================
# PART 2: EVALUATION FUNCTIONS
# ============================================================================

def calculate_metrics(y_true, y_pred, y_proba):
    """Calculate comprehensive classification metrics with macro averaging"""
    metrics = {
        'roc_auc': float(roc_auc_score(y_true, y_proba)),
        'accuracy': float(accuracy_score(y_true, y_pred)),
        'precision_macro': float(precision_score(y_true, y_pred, average='macro', zero_division=0)),
        'recall_macro': float(recall_score(y_true, y_pred, average='macro', zero_division=0)),
        'f1_macro': float(f1_score(y_true, y_pred, average='macro', zero_division=0)),
        'log_loss': float(log_loss(y_true, y_proba, labels=[1, 2]))
    }
    return metrics

# ============================================================================
# PART 3: TEST SET EVALUATION
# ============================================================================

# ============================================================================
# PART 3: TEST SET EVALUATION
# ============================================================================

print("="*80)
print("TESTING PHASE EXECUTION")
print("="*80)
print()

test_results = {'runs': []}
start_time = time.time()

for run_id in range(n_runs):
    run_info = training_results['runs'][run_id]

    print(f"Evaluating Model {run_id + 1}/{n_runs}")
    print(f"  Run ID: {run_info['run_id']}")
    print(f"  Random Seed: {run_info['random_seed']}")
    print(f"  Features: {run_info['n_features']}")

    # Load trained model
    model_path = Path(run_info['model_path'])

    if not model_path.exists():
        raise FileNotFoundError(f"Model file not found: {model_path}")

    model = xgb.XGBClassifier()
    model.load_model(str(model_path))

    # Get selected features for this run
    selected_features = run_info['selected_features']

    # Transform test set with selected features
    X_test_selected = X_test[selected_features]

    # Predict on test set (consistent with training)
    y_test_proba = model.predict_proba(X_test_selected)[:, 1]
    y_test_pred = model.predict(X_test_selected)

    # Calculate test metrics (MACRO AVERAGING)
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)

    print(f"  Test AUC: {test_metrics['roc_auc']:.4f}")
    print(f"  Test Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"  Test F1 (macro): {test_metrics['f1_macro']:.4f}")
    print()

    # Store results
    test_results['runs'].append({
        'run_id': run_id,
        'random_seed': run_info['random_seed'],
        'n_features': run_info['n_features'],
        'selected_features': selected_features,
        'validation_auc': run_info['validation_metrics']['roc_auc'],
        'test_metrics': test_metrics
    })

execution_time = time.time() - start_time

print("="*80)
print("ALL MODELS EVALUATED")
print("="*80)
print(f"Total Testing Time: {execution_time:.2f} seconds")
print()

# ============================================================================
# PART 4: STATISTICAL SUMMARY
# ============================================================================

print("="*80)
print("STATISTICAL SUMMARY")
print("="*80)
print()

# Extract test metrics
test_aucs = [run['test_metrics']['roc_auc'] for run in test_results['runs']]
test_accuracies = [run['test_metrics']['accuracy'] for run in test_results['runs']]
test_precisions = [run['test_metrics']['precision_macro'] for run in test_results['runs']]
test_recalls = [run['test_metrics']['recall_macro'] for run in test_results['runs']]
test_f1_scores = [run['test_metrics']['f1_macro'] for run in test_results['runs']]
test_log_losses = [run['test_metrics']['log_loss'] for run in test_results['runs']]

# Calculate statistics
statistics = {
    'test_auc_mean': float(np.mean(test_aucs)),
    'test_auc_std': float(np.std(test_aucs)),
    'test_auc_min': float(np.min(test_aucs)),
    'test_auc_max': float(np.max(test_aucs)),
    'test_auc_ci_lower': float(np.percentile(test_aucs, 2.5)),
    'test_auc_ci_upper': float(np.percentile(test_aucs, 97.5)),
    'test_accuracy_mean': float(np.mean(test_accuracies)),
    'test_accuracy_std': float(np.std(test_accuracies)),
    'test_precision_macro_mean': float(np.mean(test_precisions)),
    'test_precision_macro_std': float(np.std(test_precisions)),
    'test_recall_macro_mean': float(np.mean(test_recalls)),
    'test_recall_macro_std': float(np.std(test_recalls)),
    'test_f1_macro_mean': float(np.mean(test_f1_scores)),
    'test_f1_macro_std': float(np.std(test_f1_scores)),
    'test_log_loss_mean': float(np.mean(test_log_losses)),
    'test_log_loss_std': float(np.std(test_log_losses)),
    'best_run_index': int(np.argmax(test_aucs))
}

print("Test Set Performance (Macro Averaging):")
print()
print(f"  AUC:")
print(f"    Mean ± Std: {statistics['test_auc_mean']:.4f} ± {statistics['test_auc_std']:.4f}")
print(f"    95% CI: [{statistics['test_auc_ci_lower']:.4f}, {statistics['test_auc_ci_upper']:.4f}]")
print(f"    Range: [{statistics['test_auc_min']:.4f}, {statistics['test_auc_max']:.4f}]")
print()
print(f"  Accuracy:")
print(f"    Mean ± Std: {statistics['test_accuracy_mean']:.4f} ± {statistics['test_accuracy_std']:.4f}")
print()
print(f"  Precision (macro):")
print(f"    Mean ± Std: {statistics['test_precision_macro_mean']:.4f} ± {statistics['test_precision_macro_std']:.4f}")
print()
print(f"  Recall (macro):")
print(f"    Mean ± Std: {statistics['test_recall_macro_mean']:.4f} ± {statistics['test_recall_macro_std']:.4f}")
print()
print(f"  F1-Score (macro):")
print(f"    Mean ± Std: {statistics['test_f1_macro_mean']:.4f} ± {statistics['test_f1_macro_std']:.4f}")
print()
print(f"  Log Loss:")
print(f"    Mean ± Std: {statistics['test_log_loss_mean']:.4f} ± {statistics['test_log_loss_std']:.4f}")
print()

best_run_idx = statistics['best_run_index']
best_run = test_results['runs'][best_run_idx]

print("Best Performing Model:")
print(f"  Run ID: {best_run['run_id']}")
print(f"  Test AUC: {best_run['test_metrics']['roc_auc']:.4f}")
print(f"  Test Accuracy: {best_run['test_metrics']['accuracy']:.4f}")
print(f"  Test F1 (macro): {best_run['test_metrics']['f1_macro']:.4f}")
print(f"  Features: {best_run['n_features']}")
print()

# ============================================================================
# PART 5: VALIDATION-TEST CONSISTENCY ANALYSIS
# ============================================================================

print("="*80)
print("VALIDATION-TEST CONSISTENCY")
print("="*80)
print()

val_aucs = [run['validation_auc'] for run in test_results['runs']]
val_test_gaps = [val_auc - test_auc for val_auc, test_auc in zip(val_aucs, test_aucs)]

consistency_stats = {
    'mean_val_test_gap': float(np.mean(val_test_gaps)),
    'std_val_test_gap': float(np.std(val_test_gaps)),
    'correlation': float(np.corrcoef(val_aucs, test_aucs)[0, 1])
}

print("Validation-Test AUC Gap:")
print(f"  Mean Gap: {consistency_stats['mean_val_test_gap']:.4f}")
print(f"  Std Gap: {consistency_stats['std_val_test_gap']:.4f}")
print(f"  Correlation: {consistency_stats['correlation']:.4f}")
print()

if abs(consistency_stats['mean_val_test_gap']) < 0.03:
    print("  Status: Good consistency (gap < 0.03)")
elif abs(consistency_stats['mean_val_test_gap']) < 0.05:
    print("  Status: Acceptable consistency (gap < 0.05)")
else:
    print("  Warning: Large validation-test gap detected")
print()

# ============================================================================
# PART 6: VISUALIZATIONS
# ============================================================================

print("="*80)
print("GENERATING VISUALIZATIONS")
print("="*80)
print()

viz_dir = OUTPUT_DIR / 'visualizations'
viz_dir.mkdir(exist_ok=True)

# Visualization 1: Test Performance Distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].boxplot(test_aucs)
axes[0].set_ylabel('Test AUC')
axes[0].set_title('Test AUC Distribution')
axes[0].grid(alpha=0.3)

axes[1].boxplot(test_accuracies)
axes[1].set_ylabel('Test Accuracy')
axes[1].set_title('Test Accuracy Distribution')
axes[1].grid(alpha=0.3)

axes[2].boxplot(test_f1_scores)
axes[2].set_ylabel('Test F1-Score (macro)')
axes[2].set_title('Test F1-Score Distribution')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(viz_dir / 'test_performance_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

print(f"  Saved: test_performance_distribution.png")

# Visualization 2: Validation vs Test AUC
fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(val_aucs, test_aucs, s=100, alpha=0.6, edgecolors='black')
ax.plot([min(val_aucs), max(val_aucs)], [min(val_aucs), max(val_aucs)],
        'r--', linewidth=2, label='Perfect Alignment')

for i, run in enumerate(test_results['runs']):
    ax.annotate(f"R{i}", (val_aucs[i], test_aucs[i]), fontsize=8)

ax.set_xlabel('Validation AUC', fontsize=12)
ax.set_ylabel('Test AUC', fontsize=12)
ax.set_title(f'Validation vs Test AUC (Correlation: {consistency_stats["correlation"]:.3f})',
             fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(viz_dir / 'validation_vs_test_auc.png', dpi=300, bbox_inches='tight')
plt.close()

print(f"  Saved: validation_vs_test_auc.png")

# Visualization 3: Confusion Matrix for Best Model
best_model_path = Path(training_results['runs'][best_run_idx]['model_path'])
best_model = XGBClassifier()
best_model.load_model(str(best_model_path))
X_test_best = X_test[best_run['selected_features']]
y_test_proba_best = best_model.predict_proba(X_test_best)[:, 1]
y_test_pred_best = best_model.predict(X_test_best)

cm = confusion_matrix(y_test, y_test_pred_best)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Non-ASD', 'ASD'], yticklabels=['Non-ASD', 'ASD'])
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('Actual', fontsize=12)
ax.set_title(f'Confusion Matrix - Best Model (Run {best_run_idx})\nTest AUC: {best_run["test_metrics"]["roc_auc"]:.4f}',
             fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(viz_dir / 'test_confusion_matrix_best.png', dpi=300, bbox_inches='tight')
plt.close()

print(f"  Saved: test_confusion_matrix_best.png")

# Visualization 4: ROC Curve for Best Model
fpr, tpr, _ = roc_curve(y_test, y_test_proba_best)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, color='blue', linewidth=2.5,
        label=f'GWO-XGBoost (AUC={best_run["test_metrics"]["roc_auc"]:.4f})')
ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('ROC Curve - Best Model', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(viz_dir / 'test_roc_curve_best.png', dpi=300, bbox_inches='tight')
plt.close()

print(f"  Saved: test_roc_curve_best.png")
print()

# ============================================================================
# PART 7: SAVE TEST RESULTS
# ============================================================================

print("="*80)
print("SAVING TEST RESULTS")
print("="*80)
print()

test_results_data = {
    'experiment_info': {
        'name': 'GWO-XGBoost Testing Phase',
        'description': 'Unbiased evaluation on held-out test set',
        'timestamp': datetime.now().isoformat(),
        'n_runs': n_runs,
        'total_runtime_seconds': execution_time
    },
    'runs': test_results['runs'],
    'statistics': statistics,
    'consistency_analysis': consistency_stats,
    'best_model': {
        'run_id': best_run['run_id'],
        'test_auc': best_run['test_metrics']['roc_auc'],
        'all_test_metrics': best_run['test_metrics'],
        'n_features': best_run['n_features'],
        'selected_features': best_run['selected_features']
    }
}

results_file = OUTPUT_DIR / 'gwo_test_results.json'
with open(results_file, 'w') as f:
    json.dump(test_results_data, f, indent=2)

print(f"Test results saved to: {results_file}")
print()

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("="*80)
print("TESTING PHASE COMPLETED SUCCESSFULLY")
print("="*80)
print()

print("Summary:")
print(f"  Models Evaluated: {n_runs}")
print(f"  Test AUC: {statistics['test_auc_mean']:.4f} ± {statistics['test_auc_std']:.4f}")
print(f"  Test F1 (macro): {statistics['test_f1_macro_mean']:.4f} ± {statistics['test_f1_macro_std']:.4f}")
print(f"  Best Model AUC: {best_run['test_metrics']['roc_auc']:.4f}")
print(f"  Validation-Test Correlation: {consistency_stats['correlation']:.4f}")
print()

print("Output Files:")
print(f"  1. Test Results: {results_file}")
print(f"  2. Visualizations: {viz_dir} (4 plots)")
print()

print("All metrics use macro averaging for precision, recall, and F1-score.")
print()

print("Proceed to Cell 7 for Feature Stability & Baseline Alignment analysis.")
print("="*80)

TESTING PHASE INITIALIZATION

Training Directory: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_5_final_run_training
Output Directory: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_6_final_run_testing

Loading training results from Cell 5...
  Number of trained models: 10

Loading test set...
  Test Set: (397, 25) (samples, features)
  Target Distribution: {np.int64(0): np.int64(182), np.int64(1): np.int64(215)}

TESTING PHASE EXECUTION

Evaluating Model 1/10
  Run ID: 0
  Random Seed: 0
  Features: 8
  Test AUC: 0.9941
  Test Accuracy: 0.9521
  Test F1 (macro): 0.9514

Evaluating Model 2/10
  Run ID: 1
  Random Seed: 1
  Features: 10
  Test AUC: 0.9981
  Test Accuracy: 0.9597
  Test F1 (macro): 0.9591

Evaluating Model 3/10
  Run ID: 2
  Random Seed: 2
  Features: 8
  Test AUC: 0.9962
  Test Accuracy: 0.9547
  Test F1 (macro): 0.9540

Evaluating Model 4/10
  Run ID: 3
  Random Seed: 3
  Features: 13
  Test AUC: 0.

In [ ]:
# @title Cell 7: Feature Stability & Baseline Alignment

import json
import os
from datetime import datetime
from pathlib import Path
import numpy as np
from typing import Dict, List, Tuple, Any

# ============================================================================
# CONFIGURATION
# ============================================================================

CONFIG = {
    'gwo_results_path': '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_5_final_run_training/gwo_training_results.json',
    'baseline1_path': '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_02/cell_2_baseline1/training/baseline_1_training_results.json',
    'baseline2_path': '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_02/cell_3_baseline2/training/baseline_2_training_results.json',
    'baseline3_path': '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_02/cell_4_baseline3/training/baseline_3_training_results.json',
    'output_dir': '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_7_featureStability_baselineAlignment/',
    'stable_threshold': 0.8,
    'core_threshold': 1.0
}

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def load_json(filepath: str) -> Dict:
    """Load JSON file with error handling."""
    try:
        with open(filepath, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found - {filepath}")
        raise
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format - {filepath}")
        raise

def save_json(data: Dict, filepath: str) -> None:
    """Save JSON file with pretty formatting."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"Saved: {filepath}")

def calculate_jaccard_similarity(set1: List[str], set2: List[str]) -> Dict[str, Any]:
    """Calculate Jaccard similarity between two feature sets."""
    s1 = set(set1)
    s2 = set(set2)

    intersection = s1 & s2
    union = s1 | s2

    if len(union) == 0:
        jaccard = 0.0
    else:
        jaccard = len(intersection) / len(union)

    return {
        'jaccard_similarity': round(jaccard, 4),
        'intersection_count': len(intersection),
        'union_count': len(union),
        'common_features': sorted(list(intersection)),
        'set1_unique': sorted(list(s1 - s2)),
        'set2_unique': sorted(list(s2 - s1))
    }

# ============================================================================
# GWO FEATURE STABILITY ANALYSIS
# ============================================================================

def analyze_gwo_stability(gwo_results: Dict) -> Dict[str, Any]:
    """Analyze feature selection stability across GWO runs."""

    # Access runs from nested structure
    runs = gwo_results.get('training_results', gwo_results).get('runs', [])
    n_runs = len(runs)

    # Collect all selected features across runs
    all_selected_features = []
    feature_counts = {}
    n_features_per_run = []

    for run in runs:
        selected = run['selected_features']
        all_selected_features.append(selected)
        n_features_per_run.append(len(selected))

        for feature in selected:
            feature_counts[feature] = feature_counts.get(feature, 0) + 1

    # Calculate selection frequency
    selection_frequency = {
        feature: count / n_runs
        for feature, count in feature_counts.items()
    }

    # Identify feature tiers
    core_features = [
        f for f, freq in selection_frequency.items()
        if freq >= CONFIG['core_threshold']
    ]

    stable_features = [
        f for f, freq in selection_frequency.items()
        if freq >= CONFIG['stable_threshold']
    ]

    # Statistics
    stats = {
        'n_features_mean': float(np.mean(n_features_per_run)),
        'n_features_std': float(np.std(n_features_per_run)),
        'n_features_min': int(np.min(n_features_per_run)),
        'n_features_max': int(np.max(n_features_per_run)),
        'n_features_range': [int(np.min(n_features_per_run)), int(np.max(n_features_per_run))]
    }

    return {
        'n_runs': n_runs,
        'selection_frequency': {k: round(v, 3) for k, v in sorted(selection_frequency.items(), key=lambda x: x[1], reverse=True)},
        'core_features': sorted(core_features),
        'stable_features': sorted(stable_features),
        'representative_features': sorted(stable_features),
        'statistics': stats,
        'tier_counts': {
            'core_100percent': len(core_features),
            'stable_80percent': len(stable_features),
            'total_unique_features': len(feature_counts)
        }
    }

# ============================================================================
# BASELINE FEATURE EXTRACTION
# ============================================================================

def extract_baseline2_features(baseline2_results: Dict) -> Dict[str, Any]:
    """Extract selected features from SelectKBest (deterministic method)."""

    # Access feature_selection from nested structure
    feature_selection = baseline2_results.get('feature_selection', {})

    # If not found at top level, check inside training_results
    if not feature_selection:
        training_results = baseline2_results.get('training_results', {})
        feature_selection = training_results.get('feature_selection', {})

    return {
        'method_name': 'Baseline 2: XGBoost with SelectKBest',
        'method_type': 'deterministic',
        'n_features': feature_selection.get('n_features_selected', 0),
        'selected_features': feature_selection.get('selected_features', []),
        'selection_method': 'Filter method - ANOVA F-test'
    }

def extract_baseline3_features(baseline3_results: Dict) -> Dict[str, Any]:
    """Extract stable features from RFECV (stochastic method)."""

    # Access runs from nested structure
    training_results = baseline3_results.get('training_results', baseline3_results)
    runs = training_results.get('runs', [])
    n_runs = len(runs)

    # Collect features across runs
    feature_counts = {}
    n_features_per_run = []

    for run in runs:
        rfecv_results = run.get('rfecv_results', {})
        selected = rfecv_results.get('selected_features', [])
        n_features_per_run.append(len(selected))

        for feature in selected:
            feature_counts[feature] = feature_counts.get(feature, 0) + 1

    # Calculate selection frequency
    selection_frequency = {
        feature: count / n_runs
        for feature, count in feature_counts.items()
    }

    # Stable features (≥80% selection)
    stable_features = [
        f for f, freq in selection_frequency.items()
        if freq >= CONFIG['stable_threshold']
    ]

    return {
        'method_name': 'Baseline 3: XGBoost with RFECV',
        'method_type': 'stochastic',
        'n_runs': n_runs,
        'n_features_mean': float(np.mean(n_features_per_run)),
        'n_features_std': float(np.std(n_features_per_run)),
        'stable_features': sorted(stable_features),
        'n_stable_features': len(stable_features),
        'selection_method': 'Wrapper method - Recursive feature elimination with CV'
    }

# ============================================================================
# BASELINE ALIGNMENT ANALYSIS
# ============================================================================

def analyze_baseline_alignment(gwo_stability: Dict, baseline2_info: Dict, baseline3_info: Dict) -> Dict[str, Any]:
    """Calculate Jaccard similarity between GWO and baselines."""

    gwo_features = gwo_stability['representative_features']

    # Comparison with Baseline 2 (SelectKBest)
    baseline2_comparison = calculate_jaccard_similarity(
        gwo_features,
        baseline2_info['selected_features']
    )

    # Comparison with Baseline 3 (RFECV stable features)
    baseline3_comparison = calculate_jaccard_similarity(
        gwo_features,
        baseline3_info['stable_features']
    )

    return {
        'gwo_representative_features': {
            'features': gwo_features,
            'count': len(gwo_features),
            'definition': 'Stable features with ≥80% selection frequency across 10 runs'
        },
        'baseline_2_selectkbest': {
            'method_info': {
                'name': baseline2_info['method_name'],
                'method_type': baseline2_info['method_type'],
                'n_features': baseline2_info['n_features']
            },
            'features': baseline2_info['selected_features'],
            'comparison_with_gwo': baseline2_comparison
        },
        'baseline_3_rfecv': {
            'method_info': {
                'name': baseline3_info['method_name'],
                'method_type': baseline3_info['method_type'],
                'n_features_stable': baseline3_info['n_stable_features'],
                'n_features_mean': baseline3_info['n_features_mean'],
                'n_features_std': baseline3_info['n_features_std']
            },
            'features': baseline3_info['stable_features'],
            'comparison_with_gwo': baseline3_comparison
        }
    }

# ============================================================================
# SUMMARY INSIGHTS
# ============================================================================

def generate_summary_insights(gwo_stability: Dict, alignment: Dict) -> Dict[str, Any]:
    """Generate high-level summary insights."""

    # Find highest agreement
    baseline2_jaccard = alignment['baseline_2_selectkbest']['comparison_with_gwo']['jaccard_similarity']
    baseline3_jaccard = alignment['baseline_3_rfecv']['comparison_with_gwo']['jaccard_similarity']

    if baseline3_jaccard > baseline2_jaccard:
        highest_agreement = {
            'method': 'Baseline 3: RFECV',
            'jaccard_similarity': baseline3_jaccard
        }
    else:
        highest_agreement = {
            'method': 'Baseline 2: SelectKBest',
            'jaccard_similarity': baseline2_jaccard
        }

    # Core features validation
    core_features_validation = {}
    for feature in gwo_stability['core_features']:
        in_baseline2 = feature in alignment['baseline_2_selectkbest']['features']
        in_baseline3 = feature in alignment['baseline_3_rfecv']['features']

        if in_baseline2 and in_baseline3:
            status = "Selected by all methods"
        elif in_baseline2 or in_baseline3:
            methods = []
            if in_baseline2:
                methods.append("SelectKBest")
            if in_baseline3:
                methods.append("RFECV")
            status = f"Selected by GWO + {', '.join(methods)}"
        else:
            status = "GWO unique selection"

        core_features_validation[feature] = status

    # Feature reduction effectiveness
    baseline2_features = alignment['baseline_2_selectkbest']['method_info']['n_features']
    baseline3_features = alignment['baseline_3_rfecv']['method_info']['n_features_stable']
    gwo_features = len(gwo_stability['representative_features'])

    reduction_vs_baseline2 = round((1 - gwo_features / baseline2_features) * 100, 1)
    reduction_vs_baseline3 = round((1 - gwo_features / baseline3_features) * 100, 1)

    return {
        'highest_agreement': highest_agreement,
        'core_features_validation': core_features_validation,
        'feature_reduction_effectiveness': {
            'gwo_features': gwo_features,
            'vs_selectkbest': f"{reduction_vs_baseline2}% reduction ({baseline2_features}→{gwo_features})",
            'vs_rfecv': f"{reduction_vs_baseline3}% reduction ({baseline3_features}→{gwo_features})"
        },
        'alignment_summary': {
            'jaccard_vs_selectkbest': baseline2_jaccard,
            'jaccard_vs_rfecv': baseline3_jaccard,
            'mean_jaccard': round((baseline2_jaccard + baseline3_jaccard) / 2, 4)
        }
    }

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution function for Cell 7."""

    print("=" * 80)
    print("CELL 7: FEATURE STABILITY & BASELINE ALIGNMENT ANALYSIS")
    print("=" * 80)
    print()

    # Create output directory
    os.makedirs(CONFIG['output_dir'], exist_ok=True)

    # Step 1: Load all results
    print("Step 1: Loading experimental results...")
    gwo_results = load_json(CONFIG['gwo_results_path'])
    baseline2_results = load_json(CONFIG['baseline2_path'])
    baseline3_results = load_json(CONFIG['baseline3_path'])
    print("All results loaded successfully")
    print()

    # Step 2: Analyze GWO feature stability
    print("Step 2: Analyzing GWO feature stability...")
    gwo_stability = analyze_gwo_stability(gwo_results)
    print(f"  - Core features (100%): {len(gwo_stability['core_features'])}")
    print(f"  - Stable features (≥80%): {len(gwo_stability['stable_features'])}")
    print(f"  - Mean features per run: {gwo_stability['statistics']['n_features_mean']:.2f} ± {gwo_stability['statistics']['n_features_std']:.2f}")
    print()

    # Step 3: Extract baseline features
    print("Step 3: Extracting baseline features...")
    baseline2_info = extract_baseline2_features(baseline2_results)
    baseline3_info = extract_baseline3_features(baseline3_results)
    print(f"  - Baseline 2 (SelectKBest): {baseline2_info['n_features']} features")
    print(f"  - Baseline 3 (RFECV stable): {baseline3_info['n_stable_features']} features")
    print()

    # Step 4: Analyze alignment
    print("Step 4: Computing baseline alignment...")
    alignment = analyze_baseline_alignment(gwo_stability, baseline2_info, baseline3_info)
    jaccard_b2 = alignment['baseline_2_selectkbest']['comparison_with_gwo']['jaccard_similarity']
    jaccard_b3 = alignment['baseline_3_rfecv']['comparison_with_gwo']['jaccard_similarity']
    print(f"  - Jaccard vs SelectKBest: {jaccard_b2:.4f}")
    print(f"  - Jaccard vs RFECV: {jaccard_b3:.4f}")
    print()

    # Step 5: Generate summary
    print("Step 5: Generating summary insights...")
    summary = generate_summary_insights(gwo_stability, alignment)
    print(f"  - Highest agreement: {summary['highest_agreement']['method']}")
    print(f"  - Mean Jaccard similarity: {summary['alignment_summary']['mean_jaccard']:.4f}")
    print()

    # Step 6: Compile final results
    print("Step 6: Compiling final results...")
    final_results = {
        'experiment_info': {
            'cell_name': 'Cell 7: Feature Stability & Baseline Alignment',
            'timestamp': datetime.now().isoformat(),
            'stable_threshold': CONFIG['stable_threshold'],
            'core_threshold': CONFIG['core_threshold']
        },
        'gwo_feature_stability': gwo_stability,
        'baseline_alignment': alignment,
        'summary_insights': summary
    }

    # Step 7: Save results
    print("Step 7: Saving results...")
    output_path = os.path.join(CONFIG['output_dir'], 'feature_stability_baseline_alignment.json')
    save_json(final_results, output_path)
    print()

    # Final summary
    print("=" * 80)
    print("ANALYSIS COMPLETE")
    print("=" * 80)
    print(f"GWO Stable Features: {gwo_stability['representative_features']}")
    print(f"Jaccard vs SelectKBest: {jaccard_b2:.4f}")
    print(f"Jaccard vs RFECV: {jaccard_b3:.4f}")
    print(f"Output saved to: {CONFIG['output_dir']}")
    print()

# ============================================================================
# EXECUTE
# ============================================================================

if __name__ == "__main__":
    main()

CELL 7: FEATURE STABILITY & BASELINE ALIGNMENT ANALYSIS

Step 1: Loading experimental results...
All results loaded successfully

Step 2: Analyzing GWO feature stability...
  - Core features (100%): 4
  - Stable features (≥80%): 5
  - Mean features per run: 9.40 ± 2.42

Step 3: Extracting baseline features...
  - Baseline 2 (SelectKBest): 20 features
  - Baseline 3 (RFECV stable): 25 features

Step 4: Computing baseline alignment...
  - Jaccard vs SelectKBest: 0.1905
  - Jaccard vs RFECV: 0.2000

Step 5: Generating summary insights...
  - Highest agreement: Baseline 3: RFECV
  - Mean Jaccard similarity: 0.1953

Step 6: Compiling final results...
Step 7: Saving results...
Saved: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_7_featureStability_baselineAlignment/feature_stability_baseline_alignment.json

ANALYSIS COMPLETE
GWO Stable Features: ['A1', 'Age_Years', 'Family_mem_with_ASD', 'Qchat_10_Score', 'Sex']
Jaccard vs SelectKBest: 0.1905
Jaccard vs RF

In [ ]:
# @title Cell 8: GWO-XGBoost Feature Importance Extraction
"""
==========================================
Extract and aggregate feature importance scores from 10 GWO runs
to validate that selected features have high predictive value.

Author: Generated for ASD GWO-XGBoost Research
Date: 2025-10-28
"""

import json
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import xgboost as xgb
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

# Input paths
GWO_RESULTS_PATH = '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_6_final_run_testing/gwo_test_results.json'
TRAIN_DATA_PATH = '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/01_Dataset/splits/no_ethnicity/preprocessed/train_set_preprocessed.csv'
VAL_DATA_PATH = '/content/drive/MyDrive/ASD_GWO_XGBoost_Project/01_Dataset/splits/no_ethnicity/preprocessed/val_set_preprocessed.csv'

# Output directory
OUTPUT_DIR = Path('/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_8_features_importance_gwo')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# XGBoost configuration (matching GWO final training config)
# Note: Using 'hist' for compatibility (GPU auto-detected if available)
XGBOOST_CONFIG = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'n_estimators': 500,
    'max_depth': 8,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'hist',  # Universal method (auto-uses GPU if available)
    'random_state': 42
}

# ============================================================================
# LOAD DATA
# ============================================================================

print("="*80)
print("FEATURE IMPORTANCE EXTRACTION FOR GWO-XGBOOST")
print("="*80)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

print("Step 1: Loading GWO results...")
with open(GWO_RESULTS_PATH, 'r') as f:
    gwo_results = json.load(f)

print(f"✓ Loaded results for {len(gwo_results['runs'])} runs")
print(f"  - Mean features selected: {gwo_results['statistics']['test_auc_mean']:.4f}")

print("\nStep 2: Loading preprocessed datasets...")
train_df = pd.read_csv(TRAIN_DATA_PATH)
val_df = pd.read_csv(VAL_DATA_PATH)

print(f"✓ Train set: {train_df.shape[0]} samples, {train_df.shape[1]-1} features")
print(f"✓ Val set: {val_df.shape[0]} samples, {val_df.shape[1]-1} features")

# Check available columns
print("\nAvailable columns in train dataset:")
print(train_df.columns.tolist())
print(f"\nTotal columns: {len(train_df.columns)}")

# Auto-detect target column (try common variations)
possible_target_names = ['ASD_traits', 'Class_ASD', 'ASD', 'target', 'label', 'y']
target_column = None

for col_name in possible_target_names:
    if col_name in train_df.columns:
        target_column = col_name
        print(f"\n✓ Target column detected: '{target_column}'")
        break

if target_column is None:
    # If not found in common names, assume last column is target
    target_column = train_df.columns[-1]
    print(f"\n⚠ Target column not found in common names, using last column: '{target_column}'")

# Separate features and target
X_train = train_df.drop(target_column, axis=1)
y_train = train_df[target_column]
X_val = val_df.drop(target_column, axis=1)
y_val = val_df[target_column]

print(f"\n✓ Feature matrix: {X_train.shape}")
print(f"✓ Target distribution (train): {y_train.value_counts().to_dict()}")

# ============================================================================
# EXTRACT FEATURE IMPORTANCE FROM EACH RUN
# ============================================================================

print("\n" + "="*80)
print("Step 3: Extracting feature importance from each run...")
print("="*80)

importance_by_run = []
importance_by_feature = defaultdict(list)

for run_data in gwo_results['runs']:
    run_id = run_data['run_id']
    selected_features = run_data['selected_features']
    n_features = len(selected_features)

    print(f"\nRun {run_id}: {n_features} features selected")
    print(f"  Features: {', '.join(selected_features[:5])}{'...' if n_features > 5 else ''}")

    # Prepare data with selected features only
    X_train_selected = X_train[selected_features]
    X_val_selected = X_val[selected_features]

    # Train XGBoost model with run-specific random state
    run_config = XGBOOST_CONFIG.copy()
    run_config['random_state'] = XGBOOST_CONFIG['random_state'] + run_id

    model = xgb.XGBClassifier(**run_config)

    model.fit(
        X_train_selected, y_train,
        eval_set=[(X_val_selected, y_val)],
        verbose=False
    )

    # Extract feature importance (gain-based)
    importance_dict = model.get_booster().get_score(importance_type='gain')

    # Normalize to sum = 1.0
    total_importance = sum(importance_dict.values())
    # Handle case where all importances are zero (shouldn't happen with gain, but safety check)
    if total_importance > 0:
        normalized_importance = {
            feat: imp / total_importance
            for feat, imp in importance_dict.items()
        }
    else:
        normalized_importance = {feat: 0.0 for feat in importance_dict.keys()}


    # Map feature names (XGBoost uses f0, f1, ... naming)
    feature_importance = {}
    # Need to map the feature names correctly based on the columns passed to the model
    # The feature_names in the model's importance are 'f0', 'f1', ... corresponding to the order
    # of columns in X_train_selected.
    feature_map = {f'f{i}': col_name for i, col_name in enumerate(X_train_selected.columns)}

    for xgb_feature_name, importance_value in importance_dict.items():
         actual_feature_name = feature_map.get(xgb_feature_name, xgb_feature_name) # Use xgb_feature_name if not found
         feature_importance[actual_feature_name] = normalized_importance.get(xgb_feature_name, 0.0)


    # Store run-level data
    importance_by_run.append({
        'run_id': run_id,
        'n_features': n_features,
        'selected_features': selected_features,
        'feature_importance': feature_importance,
        'validation_auc': run_data['validation_auc'],
        'test_auc': run_data['test_metrics']['roc_auc']
    })

    # Accumulate for feature-level aggregation
    for feature, importance in feature_importance.items():
        importance_by_feature[feature].append(importance)

    print(f"  ✓ Top 3 important: ", end="")
    top_3 = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:3]
    print(", ".join([f"{feat}={imp:.3f}" for feat, imp in top_3]))

# ============================================================================
# AGGREGATE FEATURE IMPORTANCE STATISTICS
# ============================================================================

print("\n" + "="*80)
print("Step 4: Aggregating feature importance statistics...")
print("="*80)

# Calculate selection frequency from GWO results
from collections import Counter
all_selected_features = []
for run in gwo_results['runs']:
    all_selected_features.extend(run['selected_features'])

feature_frequency = Counter(all_selected_features)
n_runs = len(gwo_results['runs'])

# Aggregate importance statistics
feature_statistics = {}

# Get all unique features selected across all runs
all_unique_features = set(all_selected_features)

for feature in all_unique_features:
    importance_values = importance_by_feature.get(feature, [])
    # If a feature was selected in some runs but had zero importance,
    # its list of importance_values might be empty or only contain zeros.
    # Use get to handle features that might have been selected but had no gain importance across any run.
    # Or initialize with zeros for runs where it was selected but wasn't in the top-k gain
    # This part is tricky - if a feature is selected but has 0 gain in XGBoost for that run,
    # it won't appear in get_score. Let's pad with zeros for runs where it was selected but missed.
    # This requires re-iterating through runs to be precise.

    # Revised aggregation: Go through all unique features, then find their importance in each run.
    importance_across_runs = []
    for run_data in importance_by_run:
        if feature in run_data['selected_features']:
             importance_across_runs.append(run_data['feature_importance'].get(feature, 0.0))


    if importance_across_runs:
        feature_statistics[feature] = {
            'mean_importance': float(np.mean(importance_across_runs)),
            'std_importance': float(np.std(importance_across_runs)),
            'min_importance': float(np.min(importance_across_runs)),
            'max_importance': float(np.max(importance_across_runs)),
            'selection_frequency': feature_frequency[feature] / n_runs,
            'n_times_selected': feature_frequency[feature]
        }
    else:
         # Should only happen if a feature was in selected_features but never in importance_dict, which is odd
         # Setting importance to 0.0 if it was selected but had no gain.
         if feature_frequency[feature] > 0:
              feature_statistics[feature] = {
                'mean_importance': 0.0,
                'std_importance': 0.0,
                'min_importance': 0.0,
                'max_importance': 0.0,
                'selection_frequency': feature_frequency[feature] / n_runs,
                'n_times_selected': feature_frequency[feature]
            }
         else:
             # Should not happen if iterating through all_unique_features
             feature_statistics[feature] = {
                'mean_importance': 0.0,
                'std_importance': 0.0,
                'min_importance': 0.0,
                'max_importance': 0.0,
                'selection_frequency': 0.0,
                'n_times_selected': 0
            }


# Sort by mean importance
sorted_features = sorted(
    feature_statistics.items(),
    key=lambda x: x[1]['mean_importance'],
    reverse=True
)

print(f"\n✓ Aggregated statistics for {len(feature_statistics)} unique features")
print("\nTop 10 features by mean importance:")
print("-" * 80)
print(f"{'Feature':<40} {'Mean':<10} {'Std':<10} {'Freq':<10}")
print("-" * 80)

for feature, stats in sorted_features[:10]:
    print(f"{feature:<40} {stats['mean_importance']:<10.4f} "
          f"{stats['std_importance']:<10.4f} {stats['selection_frequency']:<10.1%}")

# ============================================================================
# CATEGORIZE FEATURES
# ============================================================================

print("\n" + "="*80)
print("Step 5: Categorizing features by selection frequency...")
print("="*80)

# Core features (100% selection)
core_features = {
    feat: stats for feat, stats in feature_statistics.items()
    if stats['selection_frequency'] == 1.0
}

# Stable features (≥80%, <100%)
stable_features = {
    feat: stats for feat, stats in feature_statistics.items()
    if 0.8 <= stats['selection_frequency'] < 1.0
}

# Frequent features (≥50%, <80%)
frequent_features = {
    feat: stats for feat, stats in feature_statistics.items()
    if 0.5 <= stats['selection_frequency'] < 0.8
}

# Occasional features (<50%)
occasional_features = {
    feat: stats for feat, stats in feature_statistics.items()
    if stats['selection_frequency'] < 0.5
}

print(f"\nCore features (100% selection): {len(core_features)}")
for feat, stats in sorted(core_features.items(), key=lambda x: x[1]['mean_importance'], reverse=True):
    print(f"  • {feat:<35} importance={stats['mean_importance']:.4f}")

print(f"\nStable features (≥80%, <100%): {len(stable_features)}")
for feat, stats in sorted(stable_features.items(), key=lambda x: x[1]['mean_importance'], reverse=True):
    print(f"  • {feat:<35} importance={stats['mean_importance']:.4f} (freq={stats['selection_frequency']:.0%})")

print(f"\nFrequent features (≥50%, <80%): {len(frequent_features)}")
print(f"Occasional features (<50%): {len(occasional_features)}")

# ============================================================================
# CALCULATE AGGREGATE STATISTICS
# ============================================================================

print("\n" + "="*80)
print("Step 6: Calculating aggregate statistics...")
print("="*80)

aggregate_stats = {
    'core_features': {
        'count': len(core_features),
        'mean_importance': float(np.mean([s['mean_importance'] for s in core_features.values()])) if core_features else 0,
        'total_importance': float(np.sum([s['mean_importance'] for s in core_features.values()])) if core_features else 0,
        'features': list(core_features.keys())
    },
    'stable_features': {
        'count': len(stable_features),
        'mean_importance': float(np.mean([s['mean_importance'] for s in stable_features.values()])) if stable_features else 0,
        'total_importance': float(np.sum([s['mean_importance'] for s in stable_features.values()])) if stable_features else 0,
        'features': list(stable_features.keys())
    },
    'frequent_features': {
        'count': len(frequent_features),
        'mean_importance': float(np.mean([s['mean_importance'] for s in frequent_features.values()])) if frequent_features else 0,
        'total_importance': float(np.sum([s['mean_importance'] for s in frequent_features.values()])) if frequent_features else 0
    },
    'occasional_features': {
        'count': len(occasional_features),
        'mean_importance': float(np.mean([s['mean_importance'] for s in occasional_features.values()])) if occasional_features else 0,
        'total_importance': float(np.sum([s['mean_importance'] for s in occasional_features.values()])) if occasional_features else 0
    }
}

print("\nAggregate importance by category:")
print("-" * 80)
for category, stats in aggregate_stats.items():
    print(f"{category:<25} Count={stats['count']:<3} "
          f"Mean={stats['mean_importance']:.4f} "
          f"Total={stats['total_importance']:.4f}")

# ============================================================================
# SAVE RESULTS
# ============================================================================

print("\n" + "="*80)
print("Step 7: Saving results...")
print("="*80)

# Prepare final output
output_data = {
    'metadata': {
        'experiment_name': 'GWO-XGBoost Feature Importance Analysis',
        'timestamp': datetime.now().isoformat(),
        'n_runs': n_runs,
        'xgboost_config': XGBOOST_CONFIG
    },
    'importance_by_run': importance_by_run,
    'feature_statistics': {
        feat: stats for feat, stats in sorted_features
    },
    'aggregate_statistics': aggregate_stats,
    'categorized_features': {
        'core_100_percent': core_features,
        'stable_80_percent': stable_features,
        'frequent_50_percent': frequent_features,
        'occasional_below_50': {
            feat: stats for feat, stats in occasional_features.items()
        }
    }
}

# Save comprehensive JSON
output_file = OUTPUT_DIR / 'gwo_feature_importance_analysis.json'
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=2)

print(f"✓ Saved comprehensive results: {output_file}")

# Save summary CSV for easy viewing
summary_df = pd.DataFrame([
    {
        'Feature': feat,
        'Mean_Importance': stats['mean_importance'],
        'Std_Importance': stats['std_importance'],
        'Selection_Frequency': stats['selection_frequency'],
        'Times_Selected': stats['n_times_selected']
    }
    for feat, stats in sorted_features
])

summary_csv = OUTPUT_DIR / 'feature_importance_summary.csv'
summary_df.to_csv(summary_csv, index=False)
print(f"✓ Saved summary CSV: {summary_csv}")

# ============================================================================
# GENERATE KEY INSIGHTS
# ============================================================================

print("\n" + "="*80)
print("KEY INSIGHTS FOR PAPER")
print("="*80)

# Top 5 most important features overall
top_5_features = sorted_features[:5]
print("\n1. Top 5 Most Important Features (Mean Importance):")
for i, (feat, stats) in enumerate(top_5_features, 1):
    print(f"   {i}. {feat}: {stats['mean_importance']:.4f} "
          f"(selected {stats['selection_frequency']:.0%} of runs)")

# Core features contribution
core_contribution = aggregate_stats['core_features']['total_importance']
print(f"\n2. Core Features Contribution:")
print(f"   - {len(core_features)} core features contribute {core_contribution:.2%} of total importance")
print(f"   - Mean importance per core feature: {aggregate_stats['core_features']['mean_importance']:.4f}")

# Validation of GWO selection strategy
print(f"\n3. GWO Selection Strategy Validation:")
print(f"   - Core features (100% freq) have mean importance: {aggregate_stats['core_features']['mean_importance']:.4f}")
print(f"   - Occasional features (<50% freq) have mean importance: {aggregate_stats['occasional_features']['mean_importance']:.4f}")

# Check for zero division before calculating ratio
occasional_mean_importance = aggregate_stats['occasional_features']['mean_importance']
if occasional_mean_importance > 0:
    ratio = aggregate_stats['core_features']['mean_importance'] / occasional_mean_importance
    print(f"   - Ratio: {ratio:.2f}x higher")
else:
    print("   - Ratio: Cannot calculate ratio as mean importance for occasional features is zero.")


print("\n4. Suggested text for paper (Section B or C):")
print("-" * 80)
print(f'''
Feature importance analysis confirms that GWO-selected core features contribute
substantial predictive value. The {len(core_features)} universally-selected features ({', '.join(core_features.keys())}) account for {core_contribution:.1%} of total
model importance, with mean importance of {aggregate_stats['core_features']['mean_importance']:.3f},
compared to {aggregate_stats['occasional_features']['mean_importance']:.3f} for
occasionally-selected features. This validates GWO's optimization strategy of
prioritizing high-value features over low-contribution variables.
''')

print("\n" + "="*80)
print(f"ANALYSIS COMPLETE - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

FEATURE IMPORTANCE EXTRACTION FOR GWO-XGBOOST
Start time: 2025-11-23 10:14:17

Step 1: Loading GWO results...
✓ Loaded results for 10 runs
  - Mean features selected: 0.9972

Step 2: Loading preprocessed datasets...
✓ Train set: 1270 samples, 25 features
✓ Val set: 318 samples, 25 features

Available columns in train dataset:
['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10_Autism_Spectrum_Quotient', 'Social_Responsiveness_Scale', 'Age_Years', 'Qchat_10_Score', 'Speech Delay/Language Disorder', 'Learning disorder', 'Genetic_Disorders', 'Depression', 'Global developmental delay/intellectual disability', 'Social/Behavioural Issues', 'Childhood Autism Rating Scale', 'Anxiety_disorder', 'Sex', 'Jaundice', 'Family_mem_with_ASD', 'Who_completed_the_test', 'ASD_traits']

Total columns: 26

✓ Target column detected: 'ASD_traits'

✓ Feature matrix: (1270, 25)
✓ Target distribution (train): {1: 687, 0: 583}

Step 3: Extracting feature importance from each run...

Run 0: 8 features sel

In [ ]:
# @title Cell 9: Comprehensive Comparison & Statistical Analysis

"""
CELL 8 - COMPREHENSIVE COMPARISON & STATISTICAL ANALYSIS
========================================================

Objectives:
1. Load all baseline and GWO test results
2. Perform statistical significance testing (Wilcoxon signed-rank)
3. Generate 5 deliverables for IEEE publication:
   - ROC Curves Overlay
   - Feature Count Comparison Table
   - Performance Comparison Table
   - Statistical Significance Tests
   - Improvement Analysis

Input Files:
- baseline_1_test_results.json (output_notebook_02/cell_2_baseline1/testing)
- baseline_2_test_results.json (output_notebook_02/cell_3_baseline2/testing)
- baseline_3_test_results.json (output_notebook_02/cell_4_baseline3/testing)
- gwo_test_results.json (output_notebook_03/cell_6_final_run_testing)
- test_set_preprocessed.csv (01_Dataset/splits/no_ethnicity/preprocessed)

Output Directory:
- output_notebook_03/cell_9_comprehensive_results/
"""

import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wilcoxon
from sklearn.metrics import roc_curve, auc, confusion_matrix
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PART 1: CONFIGURATION & SETUP
# ============================================================================

print("=" * 80)
print("CELL 8: COMPREHENSIVE COMPARISON & STATISTICAL ANALYSIS")
print("=" * 80)
print(f"Execution started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# Define paths
BASE_PATH = '/content/drive/MyDrive/ASD_GWO_XGBoost_Project'
RESULTS_PATH = os.path.join(BASE_PATH, '03_Results')
OUTPUT_PATH = os.path.join(RESULTS_PATH, 'output_notebook_03', 'cell_9_comprehensive_results')
VIZ_PATH = os.path.join(OUTPUT_PATH, 'visualizations')

# Create output directories
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(VIZ_PATH, exist_ok=True)

# Visualization configuration (300 DPI for publication)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9

# Configuration
CONFIG = {
    'random_state': 42,
    'alpha': 0.05,  # Significance threshold
    'n_runs': 10,
    'primary_metric': 'roc_auc',
    'averaging_method': 'macro',
    'target_column': 'ASD_traits'  # Target column name
}

print("Configuration:")
print(f"  - Significance threshold (alpha): {CONFIG['alpha']}")
print(f"  - Number of runs per method: {CONFIG['n_runs']}")
print(f"  - Primary metric: {CONFIG['primary_metric']}")
print(f"  - Averaging method: {CONFIG['averaging_method']}")
print(f"  - Target column: {CONFIG['target_column']}\n")

# ============================================================================
# PART 2: LOAD ALL TEST RESULTS
# ============================================================================

print("=" * 80)
print("PART 2: LOADING TEST RESULTS")
print("=" * 80)

def load_json(filepath):
    """Load JSON file"""
    with open(filepath, 'r') as f:
        return json.load(f)

def inspect_json_structure(data, name):
    """Inspect and print JSON structure"""
    print(f"\n  Inspecting {name} structure:")
    print(f"    Top-level keys: {list(data.keys())}")

    # Check for different possible structures
    if 'runs' in data:
        print(f"    ✓ Has 'runs' key with {len(data['runs'])} items")
        return 'runs'
    elif 'test_results' in data and isinstance(data['test_results'], dict) and 'runs' in data['test_results']:
        print(f"    ✓ Has 'test_results.runs' key with {len(data['test_results']['runs'])} items")
        return 'test_results.runs'
    elif isinstance(data, list):
        print(f"    ✓ Data is a list with {len(data)} items")
        return 'list'
    else:
        print(f"    ✗ Unexpected structure")
        return None

# Define exact paths for test results
baseline_paths = {
    'baseline_1': os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_2_baseline1', 'testing', 'baseline_1_test_results.json'),
    'baseline_2': os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_3_baseline2', 'testing', 'baseline_2_test_results.json'),
    'baseline_3': os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_4_baseline3', 'testing', 'baseline_3_test_results.json'),
    'gwo': os.path.join(RESULTS_PATH, 'output_notebook_03', 'cell_6_final_run_testing', 'gwo_test_results.json')
}

print("Loading test results from:")
for method_name, path in baseline_paths.items():
    print(f"  {method_name}: {path}")
print()

all_results = {}
runs_structure = {}

for method_name, path in baseline_paths.items():
    try:
        print(f"\nLoading {method_name}...")
        data = load_json(path)
        all_results[method_name] = data

        # Inspect structure
        structure = inspect_json_structure(data, method_name)
        runs_structure[method_name] = structure

        if structure is None:
            print(f"  ERROR: Cannot determine structure for {method_name}")
            print(f"  Available keys: {list(data.keys())}")
            raise ValueError(f"Unexpected JSON structure for {method_name}")

    except FileNotFoundError:
        print(f"  ERROR: File not found - {path}")
        raise
    except Exception as e:
        print(f"  ERROR loading {method_name}: {str(e)}")
        raise

print(f"\n{'='*80}")
print(f"Successfully loaded test results for {len(all_results)} methods")
print(f"{'='*80}\n")

# ============================================================================
# PART 3: DATA EXTRACTION
# ============================================================================

print("=" * 80)
print("PART 3: DATA EXTRACTION")
print("=" * 80)

def extract_runs(data, structure_type):
    """Extract runs from different JSON structures"""
    if structure_type == 'runs':
        return data['runs']
    elif structure_type == 'test_results.runs':
        return data['test_results']['runs']
    elif structure_type == 'list':
        return data
    else:
        raise ValueError(f"Unknown structure type: {structure_type}")

# Extract metrics for all methods
metrics_data = {}
method_labels = {
    'baseline_1': 'Baseline 1: All Features',
    'baseline_2': 'Baseline 2: SelectKBest',
    'baseline_3': 'Baseline 3: RFECV',
    'gwo': 'Proposed: GWO-XGBoost'
}

for method_key, method_name in method_labels.items():
    print(f"\nExtracting metrics for {method_name}...")

    # Get runs based on structure
    runs = extract_runs(all_results[method_key], runs_structure[method_key])
    print(f"  Found {len(runs)} runs")

    # Extract all metrics
    metrics_data[method_key] = {
        'roc_auc': [],
        'accuracy': [],
        'precision_macro': [],
        'recall_macro': [],
        'f1_macro': [],
        'log_loss': []
    }

    for run in runs:
        # Handle different possible metric locations
        if 'test_metrics' in run:
            test_metrics = run['test_metrics']
        elif 'metrics' in run:
            test_metrics = run['metrics']
        else:
            print(f"  WARNING: Cannot find metrics in run. Available keys: {list(run.keys())}")
            continue

        # Extract metrics
        for metric_name in metrics_data[method_key].keys():
            if metric_name in test_metrics:
                metrics_data[method_key][metric_name].append(test_metrics[metric_name])

    # Print summary
    print(f"  Test AUC: {np.mean(metrics_data[method_key]['roc_auc']):.4f} ± {np.std(metrics_data[method_key]['roc_auc']):.4f}")
    print(f"  Accuracy: {np.mean(metrics_data[method_key]['accuracy']):.4f} ± {np.std(metrics_data[method_key]['accuracy']):.4f}")
    print(f"  F1 (macro): {np.mean(metrics_data[method_key]['f1_macro']):.4f} ± {np.std(metrics_data[method_key]['f1_macro']):.4f}")

print()

# ============================================================================
# PART 4: STATISTICAL SIGNIFICANCE TESTING
# ============================================================================

print("=" * 80)
print("PART 4: STATISTICAL SIGNIFICANCE TESTING")
print("=" * 80)

def cohens_d(group1, group2):
    """Calculate Cohen's d effect size"""
    mean_diff = np.mean(group1) - np.mean(group2)
    pooled_std = np.sqrt((np.std(group1, ddof=1)**2 + np.std(group2, ddof=1)**2) / 2)
    return mean_diff / pooled_std

def interpret_effect_size(d):
    """Interpret Cohen's d effect size"""
    d_abs = abs(d)
    if d_abs < 0.5:
        return "Small"
    elif d_abs < 0.8:
        return "Medium"
    else:
        return "Large"

# Perform Wilcoxon signed-rank tests
comparisons = [
    ('gwo', 'baseline_1', 'GWO-XGBoost vs Baseline 1'),
    ('gwo', 'baseline_2', 'GWO-XGBoost vs Baseline 2'),
    ('gwo', 'baseline_3', 'GWO-XGBoost vs Baseline 3')
]

statistical_results = []

print("\nWilcoxon Signed-Rank Tests (paired, non-parametric):\n")

for method1, method2, comparison_name in comparisons:
    aucs1 = metrics_data[method1]['roc_auc']
    aucs2 = metrics_data[method2]['roc_auc']

    # Wilcoxon test
    statistic, p_value = wilcoxon(aucs1, aucs2)

    # Cohen's d
    effect_size = cohens_d(aucs1, aucs2)
    effect_interpretation = interpret_effect_size(effect_size)

    # Statistical significance
    is_significant = p_value < CONFIG['alpha']

    result = {
        'comparison': comparison_name,
        'method1': method1,
        'method2': method2,
        'mean_auc1': float(np.mean(aucs1)),
        'mean_auc2': float(np.mean(aucs2)),
        'mean_diff': float(np.mean(aucs1) - np.mean(aucs2)),
        'p_value': float(p_value),
        'cohens_d': float(effect_size),
        'effect_size': effect_interpretation,
        'significant': bool(is_significant),  # Convert to native Python bool
        'significance_level': '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    }

    statistical_results.append(result)

    print(f"{comparison_name}:")
    print(f"  Mean AUC difference: {result['mean_diff']:.4f}")
    print(f"  p-value: {result['p_value']:.4f} {'(significant)' if is_significant else '(not significant)'}")
    print(f"  Cohen's d: {result['cohens_d']:.3f} ({effect_interpretation})")
    print()

# ============================================================================
# PART 5: DELIVERABLE 1 - ROC CURVES OVERLAY
# ============================================================================

print("=" * 80)
print("PART 5: DELIVERABLE 1 - ROC CURVES OVERLAY")
print("=" * 80)

# Load preprocessed test set
print("\nLoading preprocessed test set...")
test_set_path = os.path.join(BASE_PATH, '01_Dataset', 'splits', 'no_ethnicity',
                              'preprocessed', 'test_set_preprocessed.csv')

if not os.path.exists(test_set_path):
    print(f"ERROR: Test set not found at {test_set_path}")
    raise FileNotFoundError(f"Test set file not found: {test_set_path}")

# Load test set
test_df = pd.read_csv(test_set_path)
print(f"Loaded test set: {test_df.shape[0]} samples, {test_df.shape[1]} columns")

# Identify target column
if CONFIG['target_column'] in test_df.columns:
    target_col = CONFIG['target_column']
elif 'Class/ASD Traits' in test_df.columns:
    target_col = 'Class/ASD Traits'
elif 'Class/ASD' in test_df.columns:
    target_col = 'Class/ASD'
else:
    # Try to find any column with 'class' or 'asd' in name (case insensitive)
    possible_targets = [col for col in test_df.columns if 'class' in col.lower() or 'asd' in col.lower()]
    if possible_targets:
        target_col = possible_targets[0]
    else:
        print(f"ERROR: Cannot find target column. Available columns: {test_df.columns.tolist()}")
        raise ValueError("Target column not found")

print(f"Target column identified: {target_col}")

# Split into X and y
y_test = test_df[target_col].values
X_test = test_df.drop(columns=[target_col])

print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"Feature columns: {X_test.columns.tolist()}\n")

# Load best models and generate ROC curves
print("Generating ROC curves for best models...\n")

import xgboost as xgb

def find_best_run_index(data, structure_type):
    """Find best run index from different structures"""
    if 'statistics' in data:
        stats = data['statistics']
        if 'best_run_index' in stats:
            return stats['best_run_index']
        elif 'best_run_id' in stats:
            return stats['best_run_id']

    # Fallback: find manually
    runs = extract_runs(data, structure_type)
    best_auc = -1
    best_idx = 0

    for idx, run in enumerate(runs):
        if 'test_metrics' in run:
            auc_val = run['test_metrics'].get('roc_auc', 0)
        elif 'metrics' in run:
            auc_val = run['metrics'].get('roc_auc', 0)
        else:
            auc_val = 0

        if auc_val > best_auc:
            best_auc = auc_val
            best_idx = idx

    return best_idx

roc_data = {}
colors = {
    'baseline_1': '#1f77b4',  # blue
    'baseline_2': '#2ca02c',  # green
    'baseline_3': '#ff7f0e',  # orange
    'gwo': '#d62728'          # red
}

for method_key, method_name in method_labels.items():
    # Find best run
    best_run_idx = find_best_run_index(all_results[method_key], runs_structure[method_key])
    runs = extract_runs(all_results[method_key], runs_structure[method_key])
    best_run = runs[best_run_idx]

    print(f"Processing {method_name} (best run: {best_run_idx})...")

    # Define model paths based on method
    if method_key == 'baseline_1':
        model_path = os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_2_baseline1',
                                  'training', 'models', f'run_{best_run_idx}_model.json')
        selected_features = X_test.columns.tolist()
    elif method_key == 'baseline_2':
        model_path = os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_3_baseline2',
                                  'training', 'models', f'run_{best_run_idx}_model.json')
        # Load training results to get selected features
        training_path = os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_3_baseline2',
                                     'training', 'baseline_2_training_results.json')
        training_results = load_json(training_path)
        selected_features = training_results['feature_selection']['selected_features']
    elif method_key == 'baseline_3':
        model_path = os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_4_baseline3',
                                  'training', 'models', f'run_{best_run_idx}_model.json')
        # Load training results to get selected features for this specific run
        training_path = os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_4_baseline3',
                                     'training', 'baseline_3_training_results.json')
        training_results = load_json(training_path)
        selected_features = training_results['training_results']['runs'][best_run_idx]['rfecv_results']['selected_features']
    else:  # gwo
        model_path = os.path.join(RESULTS_PATH, 'output_notebook_03', 'cell_5_final_run_training',
                                  'models', f'run_{best_run_idx}_model.json')
        # Get selected features from best run
        if 'selected_features' in best_run:
            selected_features = best_run['selected_features']
        else:
            # Load training results
            training_path = os.path.join(RESULTS_PATH, 'output_notebook_03', 'cell_5_final_run_training',
                                         'gwo_training_results.json')
            training_results = load_json(training_path)
            selected_features = training_results['runs'][best_run_idx]['selected_features']

    # Check if model exists
    if not os.path.exists(model_path):
        print(f"  WARNING: Model not found at {model_path}")
        print(f"  Skipping ROC curve generation for {method_name}")
        continue

    # Prepare test data
    X_test_selected = X_test[selected_features]

    # Load model and predict
    try:
        model = xgb.Booster()
        model.load_model(model_path)
        dtest = xgb.DMatrix(X_test_selected)
        y_pred_proba = model.predict(dtest)

        # Calculate ROC curve
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)

        roc_data[method_key] = {
            'fpr': fpr,
            'tpr': tpr,
            'auc': roc_auc,
            'name': method_name
        }

        print(f"  {method_name}: AUC = {roc_auc:.4f}")
    except Exception as e:
        print(f"  ERROR loading model: {str(e)}")
        continue

# Plot ROC curves only if we have data
if len(roc_data) > 0:
    print("\nGenerating ROC curves plot...")

    fig, ax = plt.subplots(figsize=(8, 6))

    for method_key in ['baseline_1', 'baseline_2', 'baseline_3', 'gwo']:
        if method_key in roc_data:
            data = roc_data[method_key]
            ax.plot(data['fpr'], data['tpr'],
                    color=colors[method_key],
                    linewidth=2.5,
                    label=f"{data['name']} (AUC = {data['auc']:.3f})")

    # Plot diagonal
    ax.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Random Classifier')

    ax.set_xlabel('False Positive Rate', fontsize=11)
    ax.set_ylabel('True Positive Rate', fontsize=11)
    ax.set_title('ROC Curve Comparison - Test Set', fontsize=12, fontweight='bold')
    ax.legend(loc='lower right', frameon=True, shadow=True)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])

    plt.tight_layout()
    roc_path = os.path.join(VIZ_PATH, 'roc_comparison.png')
    plt.savefig(roc_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Saved: {roc_path}\n")
else:
    print("WARNING: No ROC data available. Skipping ROC curve generation.\n")

# ============================================================================
# PART 6: DELIVERABLE 2 - FEATURE COUNT COMPARISON TABLE
# ============================================================================

print("=" * 80)
print("PART 6: DELIVERABLE 2 - FEATURE COUNT COMPARISON TABLE")
print("=" * 80)

# Extract feature counts
feature_comparison = []

# Baseline 1
feature_comparison.append({
    'Method': 'Baseline 1: All Features',
    'N Features': '25',
    'Reduction %': '0%',
    'Selection Time (min)': '-'
})

# Baseline 2
try:
    baseline2_training_path = os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_3_baseline2',
                                           'training', 'baseline_2_training_results.json')
    baseline2_training = load_json(baseline2_training_path)
    n_features_b2 = baseline2_training['feature_selection']['n_features_selected']
    reduction_b2 = ((25 - n_features_b2) / 25) * 100
    feature_comparison.append({
        'Method': 'Baseline 2: SelectKBest',
        'N Features': str(n_features_b2),
        'Reduction %': f'{reduction_b2:.0f}%',
        'Selection Time (min)': '2.3'
    })
except Exception as e:
    print(f"WARNING: Cannot load Baseline 2 feature info: {str(e)}")
    n_features_b2 = 16
    feature_comparison.append({
        'Method': 'Baseline 2: SelectKBest',
        'N Features': 'N/A',
        'Reduction %': 'N/A',
        'Selection Time (min)': 'N/A'
    })

# Baseline 3
try:
    baseline3_training_path = os.path.join(RESULTS_PATH, 'output_notebook_02', 'cell_4_baseline3',
                                           'training', 'baseline_3_training_results.json')
    baseline3_training = load_json(baseline3_training_path)
    n_features_b3 = [run['rfecv_results']['optimal_n_features'] for run in baseline3_training['training_results']['runs']]
    mean_features_b3 = np.mean(n_features_b3)
    std_features_b3 = np.std(n_features_b3)
    reduction_b3 = ((25 - mean_features_b3) / 25) * 100
    feature_comparison.append({
        'Method': 'Baseline 3: RFECV',
        'N Features': f'{mean_features_b3:.1f} ± {std_features_b3:.1f}',
        'Reduction %': f'{reduction_b3:.0f}%',
        'Selection Time (min)': '45.2'
    })
except Exception as e:
    print(f"WARNING: Cannot load Baseline 3 feature info: {str(e)}")
    mean_features_b3 = 17.5
    std_features_b3 = 1.2
    feature_comparison.append({
        'Method': 'Baseline 3: RFECV',
        'N Features': 'N/A',
        'Reduction %': 'N/A',
        'Selection Time (min)': 'N/A'
    })

# GWO
try:
    gwo_training_path = os.path.join(RESULTS_PATH, 'output_notebook_03', 'cell_5_final_run_training',
                                     'gwo_training_results.json')
    gwo_training = load_json(gwo_training_path)
    n_features_gwo = [len(run['selected_features']) for run in gwo_training['runs']]
    mean_features_gwo = np.mean(n_features_gwo)
    std_features_gwo = np.std(n_features_gwo)
    reduction_gwo = ((25 - mean_features_gwo) / 25) * 100
    feature_comparison.append({
        'Method': 'Proposed: GWO-XGBoost',
        'N Features': f'{mean_features_gwo:.1f} ± {std_features_gwo:.1f}',
        'Reduction %': f'{reduction_gwo:.0f}%',
        'Selection Time (min)': '120*'
    })
except Exception as e:
    print(f"WARNING: Cannot load GWO feature info: {str(e)}")
    mean_features_gwo = 10.0
    std_features_gwo = 0.5
    feature_comparison.append({
        'Method': 'Proposed: GWO-XGBoost',
        'N Features': 'N/A',
        'Reduction %': 'N/A',
        'Selection Time (min)': 'N/A'
    })

feature_df = pd.DataFrame(feature_comparison)

print("\nTABLE I: Feature Selection Comparison")
print("=" * 80)
print(feature_df.to_string(index=False))
print("\n* Total time including parameter optimization\n")

# ============================================================================
# PART 7: DELIVERABLE 3 - PERFORMANCE COMPARISON TABLE
# ============================================================================

print("=" * 80)
print("PART 7: DELIVERABLE 3 - PERFORMANCE COMPARISON TABLE")
print("=" * 80)

# Build performance comparison table
performance_comparison = []

for method_key, method_name in method_labels.items():
    metrics = metrics_data[method_key]

    performance_comparison.append({
        'Method': method_name,
        'Test AUC': f"{np.mean(metrics['roc_auc']):.4f} ± {np.std(metrics['roc_auc']):.4f}",
        'Accuracy': f"{np.mean(metrics['accuracy']):.4f} ± {np.std(metrics['accuracy']):.4f}",
        'Precision (Macro)': f"{np.mean(metrics['precision_macro']):.4f} ± {np.std(metrics['precision_macro']):.4f}",
        'Recall (Macro)': f"{np.mean(metrics['recall_macro']):.4f} ± {np.std(metrics['recall_macro']):.4f}",
        'F1 (Macro)': f"{np.mean(metrics['f1_macro']):.4f} ± {np.std(metrics['f1_macro']):.4f}"
    })

performance_df = pd.DataFrame(performance_comparison)

print("\nTABLE II: Performance Comparison on Test Set")
print("=" * 80)
print(performance_df.to_string(index=False))
print("\nAll metrics use macro averaging for multi-class metrics.\n")

# ============================================================================
# PART 8: DELIVERABLE 4 - STATISTICAL SIGNIFICANCE TESTS
# ============================================================================

print("=" * 80)
print("PART 8: DELIVERABLE 4 - STATISTICAL SIGNIFICANCE TESTS")
print("=" * 80)

# Build statistical significance table
significance_comparison = []

for result in statistical_results:
    significance_comparison.append({
        'Comparison': result['comparison'],
        'p-value': f"{result['p_value']:.4f}{result['significance_level']}",
        "Cohen's d": f"{result['cohens_d']:.3f}",
        'Effect Size': result['effect_size'],
        'Conclusion': 'Significant' if result['significant'] else 'Not Significant'
    })

significance_df = pd.DataFrame(significance_comparison)

print("\nTABLE III: Statistical Significance Tests")
print("=" * 80)
print(significance_df.to_string(index=False))
print("\nSignificance levels: ** p < 0.01, * p < 0.05, ns = not significant\n")

# ============================================================================
# PART 9: DELIVERABLE 5 - IMPROVEMENT ANALYSIS
# ============================================================================

print("=" * 80)
print("PART 9: DELIVERABLE 5 - IMPROVEMENT ANALYSIS")
print("=" * 80)

# Build improvement analysis table
improvement_comparison = []

gwo_mean_auc = np.mean(metrics_data['gwo']['roc_auc'])

baseline_features = {
    'baseline_1': 25,
    'baseline_2': n_features_b2 if 'n_features_b2' in locals() else 16,
    'baseline_3': mean_features_b3
}

for method_key in ['baseline_1', 'baseline_2', 'baseline_3']:
    baseline_mean_auc = np.mean(metrics_data[method_key]['roc_auc'])
    delta_auc = gwo_mean_auc - baseline_mean_auc
    improvement_pct = (delta_auc / baseline_mean_auc) * 100

    # Feature reduction
    baseline_feat = baseline_features[method_key]
    reduction_count = baseline_feat - mean_features_gwo
    reduction_pct = ((baseline_feat - mean_features_gwo) / baseline_feat) * 100

    method_name = method_labels[method_key].replace('Baseline ', 'vs Baseline ')

    improvement_comparison.append({
        'Comparison': method_name,
        'Delta Test AUC': f"{delta_auc:+.4f}",
        'Improvement %': f"{improvement_pct:+.2f}%",
        'Feature Reduction (count)': f"{reduction_count:.1f}",
        'Feature Reduction %': f"{reduction_pct:.0f}%"
    })

improvement_df = pd.DataFrame(improvement_comparison)

print("\nTABLE IV: Performance Improvement Analysis")
print("=" * 80)
print(improvement_df.to_string(index=False))
print()

# ============================================================================
# PART 10: ADDITIONAL VISUALIZATIONS
# ============================================================================

print("=" * 80)
print("PART 10: ADDITIONAL VISUALIZATIONS")
print("=" * 80)

# Visualization 1: Performance Boxplots
print("\nGenerating performance boxplots...")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics_to_plot = ['roc_auc', 'accuracy', 'f1_macro']
metric_titles = ['Test AUC', 'Test Accuracy', 'Test F1-Score (Macro)']

for idx, (metric, title) in enumerate(zip(metrics_to_plot, metric_titles)):
    ax = axes[idx]

    data_to_plot = [metrics_data[key][metric] for key in ['baseline_1', 'baseline_2', 'baseline_3', 'gwo']]
    labels = ['B1: All', 'B2: KBest', 'B3: RFECV', 'GWO-XGB']

    bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True,
                    boxprops=dict(facecolor='lightblue', alpha=0.7),
                    medianprops=dict(color='red', linewidth=2),
                    whiskerprops=dict(linewidth=1.5),
                    capprops=dict(linewidth=1.5))

    # Highlight GWO
    bp['boxes'][3].set_facecolor('lightcoral')

    ax.set_ylabel(title, fontsize=11)
    ax.set_xlabel('Method', fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_title(title, fontsize=11, fontweight='bold')

plt.tight_layout()
boxplot_path = os.path.join(VIZ_PATH, 'performance_boxplot.png')
plt.savefig(boxplot_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"Saved: {boxplot_path}")

# Visualization 2: Feature Count Comparison
print("Generating feature count comparison...")

fig, ax = plt.subplots(figsize=(10, 6))

methods = ['Baseline 1', 'Baseline 2', 'Baseline 3', 'GWO-XGBoost']
feature_counts = [25, baseline_features['baseline_2'], mean_features_b3, mean_features_gwo]
feature_stds = [0, 0, std_features_b3, std_features_gwo]
colors_bar = ['#1f77b4', '#2ca02c', '#ff7f0e', '#d62728']

bars = ax.bar(methods, feature_counts, color=colors_bar, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.errorbar(methods, feature_counts, yerr=feature_stds, fmt='none', ecolor='black', capsize=5, linewidth=2)

# Add value labels
for bar, count, std in zip(bars, feature_counts, feature_stds):
    height = bar.get_height()
    if std > 0:
        label = f'{count:.1f}±{std:.1f}'
    else:
        label = f'{count:.0f}'
    ax.text(bar.get_x() + bar.get_width()/2., height + std + 0.5,
            label, ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.set_ylabel('Number of Features', fontsize=12)
ax.set_xlabel('Method', fontsize=12)
ax.set_title('Feature Count Comparison', fontsize=13, fontweight='bold')
ax.set_ylim([0, 30])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
feature_count_path = os.path.join(VIZ_PATH, 'feature_count_comparison.png')
plt.savefig(feature_count_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"Saved: {feature_count_path}")

# Visualization 3: Metrics Comparison Barplot
print("Generating metrics comparison barplot...")

fig, ax = plt.subplots(figsize=(12, 6))

metrics_names = ['AUC', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
metric_keys = ['roc_auc', 'accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

x = np.arange(len(metrics_names))
width = 0.2

for idx, (method_key, method_name) in enumerate(method_labels.items()):
    means = [np.mean(metrics_data[method_key][key]) for key in metric_keys]
    stds = [np.std(metrics_data[method_key][key]) for key in metric_keys]

    offset = (idx - 1.5) * width
    bars = ax.bar(x + offset, means, width, label=method_name.split(': ')[0],
                  color=colors[method_key], alpha=0.8, edgecolor='black', linewidth=0.8)

ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Performance Metrics Comparison - Test Set', fontsize=13, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics_names)
ax.legend(loc='lower right', frameon=True, shadow=True)
ax.set_ylim([0.75, 1.0])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
metrics_barplot_path = os.path.join(VIZ_PATH, 'metrics_comparison_barplot.png')
plt.savefig(metrics_barplot_path, dpi=300, bbox_inches='tight')
plt.close()

print(f"Saved: {metrics_barplot_path}\n")

# ============================================================================
# PART 11: SAVE COMPREHENSIVE RESULTS
# ============================================================================

print("=" * 80)
print("PART 11: SAVE COMPREHENSIVE RESULTS")
print("=" * 80)

comprehensive_results = {
    'metadata': {
        'experiment_name': 'Comprehensive Comparison: GWO-XGBoost vs Baselines',
        'timestamp': datetime.now().isoformat(),
        'n_methods': 4,
        'n_runs_per_method': CONFIG['n_runs'],
        'primary_metric': CONFIG['primary_metric'],
        'averaging_method': CONFIG['averaging_method']
    },
    'deliverables': {
        'roc_curves': {
            'file': 'roc_comparison.png',
            'description': 'ROC curves overlay for all methods',
            'data': {method: {'auc': float(data['auc'])} for method, data in roc_data.items()} if roc_data else {}
        },
        'feature_comparison': {
            'description': 'Feature selection comparison',
            'table': feature_comparison
        },
        'performance_comparison': {
            'description': 'Performance metrics comparison on test set',
            'table': performance_comparison
        },
        'statistical_tests': {
            'description': 'Wilcoxon signed-rank tests with effect sizes',
            'table': significance_comparison,
            'raw_results': statistical_results
        },
        'improvement_analysis': {
            'description': 'Performance improvement and feature reduction analysis',
            'table': improvement_comparison
        }
    },
    'raw_metrics': {
        method_key: {
            metric: {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values)),
                'values': [float(v) for v in values]
            } for metric, values in metrics.items()
        } for method_key, metrics in metrics_data.items()
    },
    'key_findings': [
        f"GWO-XGBoost achieved test AUC of {np.mean(metrics_data['gwo']['roc_auc']):.4f} ± {np.std(metrics_data['gwo']['roc_auc']):.4f}",
        f"Feature reduction: {reduction_gwo:.1f}% (25 → {mean_features_gwo:.1f} features)",
        f"Improvement vs Baseline 1: {improvement_comparison[0]['Delta Test AUC']} AUC ({improvement_comparison[0]['Improvement %']})",
        f"Statistical tests: {len([r for r in statistical_results if r['significant']])}/{len(statistical_results)} comparisons significant",
        f"Effect sizes: {', '.join(set([r['effect_size'] for r in statistical_results]))}"
    ],
    'output_files': {
        'visualizations': [
            'roc_comparison.png',
            'performance_boxplot.png',
            'feature_count_comparison.png',
            'metrics_comparison_barplot.png'
        ],
        'results_json': 'comprehensive_comparison.json'
    }
}

# Custom JSON encoder for numpy types
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (np.bool_, bool)):
            return bool(obj)
        return super(NumpyEncoder, self).default(obj)

# Save comprehensive results
results_json_path = os.path.join(OUTPUT_PATH, 'comprehensive_comparison.json')
try:
    with open(results_json_path, 'w') as f:
        json.dump(comprehensive_results, f, indent=2, cls=NumpyEncoder)
    print(f"\nSaved comprehensive results: {results_json_path}\n")
except Exception as e:
    print(f"\nERROR saving JSON: {str(e)}")
    print("Attempting to save with default encoder...")
    # Try to convert all values to native Python types
    import copy
    results_copy = copy.deepcopy(comprehensive_results)

    def convert_to_native(obj):
        """Recursively convert numpy types to native Python types"""
        if isinstance(obj, dict):
            return {k: convert_to_native(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_native(v) for v in obj]
        elif isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, (np.bool_, bool)):
            return bool(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj

    results_copy = convert_to_native(results_copy)

    with open(results_json_path, 'w') as f:
        json.dump(results_copy, f, indent=2)
    print(f"Successfully saved with type conversion: {results_json_path}\n")

# ============================================================================
# PART 12: GENERATE SUMMARY REPORT
# ============================================================================

print("=" * 80)
print("PART 12: SUMMARY REPORT")
print("=" * 80)

print("\n" + "=" * 80)
print("COMPREHENSIVE COMPARISON - SUMMARY REPORT")
print("=" * 80)

print("\n1. KEY FINDINGS:")
print("-" * 80)
for idx, finding in enumerate(comprehensive_results['key_findings'], 1):
    print(f"{idx}. {finding}")

print("\n2. DELIVERABLES GENERATED:")
print("-" * 80)
print("Tables:")
print("  - TABLE I: Feature Selection Comparison")
print("  - TABLE II: Performance Comparison on Test Set")
print("  - TABLE III: Statistical Significance Tests")
print("  - TABLE IV: Performance Improvement Analysis")
print("\nVisualizations:")
for viz_file in comprehensive_results['output_files']['visualizations']:
    print(f"  - {viz_file}")

print("\n3. OUTPUT FILES:")
print("-" * 80)
print(f"Results directory: {OUTPUT_PATH}")
print(f"Visualizations directory: {VIZ_PATH}")
print(f"\nAll files saved successfully!")

print("\n4. STATISTICAL SUMMARY:")
print("-" * 80)
significant_count = len([r for r in statistical_results if r['significant']])
print(f"Significant comparisons: {significant_count}/{len(statistical_results)}")
print(f"Effect sizes: {', '.join(set([r['effect_size'] for r in statistical_results]))}")
if significant_count > 0:
    print(f"Strongest improvement: {statistical_results[0]['comparison']}")

print("\n" + "=" * 80)
print(f"CELL 8 EXECUTION COMPLETED SUCCESSFULLY")
print(f"Completion time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80 + "\n")

print("Next steps:")
print("1. Review all generated tables and visualizations")
print("2. Verify statistical significance results")
print("3. Use deliverables for IEEE paper Results section")
print("4. Check comprehensive_comparison.json for complete results")

CELL 8: COMPREHENSIVE COMPARISON & STATISTICAL ANALYSIS
Execution started at: 2025-11-23 10:14:33

Configuration:
  - Significance threshold (alpha): 0.05
  - Number of runs per method: 10
  - Primary metric: roc_auc
  - Averaging method: macro
  - Target column: ASD_traits

PART 2: LOADING TEST RESULTS
Loading test results from:
  baseline_1: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_02/cell_2_baseline1/testing/baseline_1_test_results.json
  baseline_2: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_02/cell_3_baseline2/testing/baseline_2_test_results.json
  baseline_3: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_02/cell_4_baseline3/testing/baseline_3_test_results.json
  gwo: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_6_final_run_testing/gwo_test_results.json


Loading baseline_1...

  Inspecting baseline_1 structure:
    Top-level keys: ['experiment_info', 't

In [ ]:
# @title feature selection png new
import matplotlib.pyplot as plt
import numpy as np
import os

# Output directory
save_dir = "/content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_9_comprehensive_results/"
os.makedirs(save_dir, exist_ok=True)

# Data (sorted by frequency)
features = [
    'Age_Years', 'Qchat_10_Score', 'Sex', 'Family_mem_with_ASD',  # Core 100%
    'A1',  # 80%
    'A6',  # 70%
    'A7',  # 50%
    'A4', 'A8',  # 40%
    'A5', 'GDD/ID', 'CARS', 'Social/Behav',  # 30%
    'SRS', 'A2', 'Genetic', 'Learning', 'A9',  # 20%
    'A3', 'Depression', 'Anxiety', 'Jaundice'  # 10%
]

frequencies = [
    100, 100, 100, 100,  # Core
    80,  # High
    70,  # High
    50,  # Medium
    40, 40,  # Medium
    30, 30, 30, 30,  # Lower
    20, 20, 20, 20, 20,  # Lower
    10, 10, 10, 10  # Rare
]

# 🎨 Updated color scheme (matching Feature Count Comparison bar palette)
colors = []
for freq in frequencies:
    if freq == 100:
        colors.append('#E15759')  # Red
    elif freq >= 70:
        colors.append('#F28E2B')  # Orange
    elif freq >= 40:
        colors.append('#59A14F')  # Green
    else:
        colors.append('#4E79A7')  # Blue

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(10, 8))

y_pos = np.arange(len(features))
bars = ax.barh(y_pos, frequencies, color=colors, edgecolor='black', linewidth=0.8)

# Styling
ax.set_yticks(y_pos)
ax.set_yticklabels(features, fontsize=10)
ax.set_xlabel('Selection Frequency (%)', fontsize=12, fontweight='bold')
ax.set_title('Feature Selection Frequency Across 10 GWO-XGBoost Runs',
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlim(0, 105)
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Add value labels on bars
for i, (bar, freq) in enumerate(zip(bars, frequencies)):
    width = bar.get_width()
    ax.text(width + 2, bar.get_y() + bar.get_height()/2,
            f'{freq}%',
            ha='left', va='center', fontsize=9, fontweight='bold')

# Add vertical line at 100% for core features
ax.axvline(x=100, color='red', linestyle='--', linewidth=1.5, alpha=0.5)

# Add annotation for core features
ax.annotate('Core Features\n(45.2% total importance)',
            xy=(100, 1.5), xytext=(85, 8),
            bbox=dict(boxstyle='round,pad=0.5', facecolor='#E15759', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.3',
                            color='lightblue', lw=1.5),
            fontsize=9, ha='center', fontweight='bold')

plt.tight_layout()

# 💾 Save the figure to your Drive
save_path = os.path.join(save_dir, "feature_selection_frequency_new.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Figure saved to: {save_path}")


✅ Figure saved to: /content/drive/MyDrive/ASD_GWO_XGBoost_Project/03_Results/output_notebook_03/cell_9_comprehensive_results/feature_selection_frequency_new.png
